In [49]:
import requests
import env
import os
import pandas as pd
import json
from env import Client_ID
from igdb.wrapper import IGDBWrapper
import time
from datetime import datetime


#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Connecting to API

In [ ]:
url = env.get_db_url()
response = requests.post(url)
print(response.text)


In [ ]:
data = response.json()
data.keys()

In [ ]:
access_token = data['access_token']

In [ ]:
# verify access token matches
access_token == data['access_token']

In [ ]:
requests.post(url)

In [ ]:
#function to connect to api and get access token
#requests.post(url) != 200
url = env.get_db_url()
response = requests.post(url)
data = response.json()
access_token = data['access_token']


In [50]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

### Making requests from API

- Most of the requests to the API will use the POST method
- The base URL is: https://api.igdb.com/v4
- You define which endpoint you wish to query by appending /{endpoint name} to the base URL eg. https://api.igdb.com/v4/games
- Include Client ID and access_token in the HEADER of your request so that your headers look like the following. Take special care of the capitalisation. Bearer should be hard-coded.

    ''' 
    Client-ID: Client ID

    Authorization: Bearer access_token 
    '''

- You use the BODY of your request to specify the fields you want to retrieve as well as any other filters, sorting etc

**Example**

    '''
    POST https://api.igdb.com/v4/games
    Client-ID: abcdefg12345
    Authorization: Bearer access12345token
    fields *;
    '''

more examples https://api-docs.igdb.com/#examples 

In [ ]:
url = env.get_db_url()
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url)
print(response.text)

In [ ]:
url = env.get_db_url()
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url , headers=headers)
print(response.text)

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; popularity;sort popularity desc;'
r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)

In [ ]:
r.json()

In [ ]:
response.json()

In [ ]:
r.text

In [4]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; rating; sort rating asc;'
r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)

NameError: name 'access_token' is not defined

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [3]:
wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')

NameError: name 'access_token' is not defined

In [ ]:
'''With a wrapper instance already created'''
# JSON API request
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )
# parse into JSON however you like...

# Protobuf API request
from igdb.igdbapi_pb2 import GameResult
byte_array = wrapper.api_request(
            'games.pb', # Note the '.pb' suffix at the endpoint
            'fields *; limit 500;'
          )
games_message = GameResult()
games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response

In [ ]:
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )

In [ ]:
testgames = []
for i in range(0, 500):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
    testgames.append(games)

In [ ]:
testgames[0]

In [ ]:
pd.DataFrame(testgames.json())

In [ ]:
byte_array

In [ ]:
y = json.loads(byte_array)

In [ ]:
pd.DataFrame(y).head()

In [ ]:
pd.DataFrame(y).columns

In [ ]:
pd.DataFrame(y['id', 'category', 'genres', 'name', 'parent_game', 'slug','summary', 'tags', 'themes', 'url', 'websites',])

In [ ]:

pd.DataFrame(y).head(1)

In [ ]:
pd.DataFrame(y).url

In [ ]:
y

In [ ]:
games_message.ParseFromString(byte_array)

In [ ]:
type(games_message)

In [ ]:
games_message[1]

get ratings

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
#data = 'fields *; popularity;sort popularity desc;'
requests.get('https://api.igdb.com/v4/age_ratings' ,  headers=headers).text

### get games library

In [4]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

In [5]:
access_token = connect_api()

In [6]:
def connect_to_games():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; popularity;sort popularity desc;'
    r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)
    return r

In [7]:
def run_wrapper():
    wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')
    return wrapper

In [8]:
wrapper = run_wrapper()

In [10]:
# function that puts response list object into a dataframe for each page
def get_game_library(wrapper):
    game_library = pd.DataFrame()
    for i in range (0, 409):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(games)
        results_df =pd.DataFrame(y)
        game_library = pd.concat([game_library, results_df])
    return game_library

In [11]:
game_library = get_game_library(wrapper)

In [59]:
game_library.head()

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],NaN,1604620800,https://www.igdb.com/games/dogou-souken,NaN,NaN,5e7446ff-58d9-e744-f0d3-5e10241304c1,5.364576e+08,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,"[155025, 155026, 155027]","[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],NaN,1641425677,https://www.igdb.com/games/city-mysteries,NaN,NaN,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,1.404346e+09,[37],[138333],[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]","[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",[1],1643857546,https://www.igdb.com/games/sword-of-the-black-...,NaN,[84617],0b575316-96dc-b056-86e0-c958c1c95efb,1.527120e+09,[6],[188385],NaN,"[155850, 155852]",[1],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]","[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",NaN,1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,NaN,[192858],9585e273-2606-970f-3088-60261ea77068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[63465],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
game_library.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204500 entries, 0 to 499
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     204500 non-null  int64  
 1   category               204500 non-null  int64  
 2   first_release_date     117710 non-null  float64
 3   genres                 158909 non-null  object 
 4   name                   204500 non-null  object 
 5   platforms              122842 non-null  object 
 6   release_dates          122470 non-null  object 
 7   slug                   204500 non-null  object 
 8   summary                169667 non-null  object 
 9   age_ratings            44287 non-null   object 
 10  involved_companies     84563 non-null   object 
 11  game_modes             97309 non-null   object 
 12  player_perspectives    61331 non-null   object 
 13  themes                 104486 non-null  object 
 14  game_engines           16318 non-null  

,0
0,None


In [48]:
pd.DataFrame(game_library.columns)

,0
0,id
1,category
2,first_release_date
3,genres
4,name
5,platforms
6,release_dates
7,slug
8,summary
9,age_ratings


**Takeaways**

- delete  'artworks',  'cover', 'created_at', 'external_games', 'screenshots', 'similar_games',  'tags', 'updated_at', 'url', 'videos', 'websites', 'checksum',  'alternative_names',  'keywords', 'status', 'rating_count', 'storyline', 'total_rating_count',  'hypes', 'follows', 'aggregated_rating_count'

In [219]:
game_library.head(2)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],NaN,1604620800,https://www.igdb.com/games/dogou-souken,NaN,NaN,5e7446ff-58d9-e744-f0d3-5e10241304c1,536457600.0,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
game_library[game_library['genres'].isnull()].head(2)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
6,85450,NaN,0,NaN,1517405808,"[221231, 1748471, 1991656]",NaN,Transformers Prime: The Game,NaN,NaN,NaN,transformers-prime-the-game,NaN,NaN,NaN,1651604097,https://www.igdb.com/games/transformers-prime-...,NaN,NaN,50b54ee2-8d2c-1231-750e-161a1ff246cc,NaN,NaN,NaN,[67799],[85230],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,104748,NaN,0,NaN,1530489600,"[1155919, 1972586]",NaN,Space station - build your own ISS,NaN,NaN,NaN,space-station-build-your-own-iss,NaN,NaN,NaN,1604620800,https://www.igdb.com/games/space-station-build...,NaN,NaN,d49fe10f-a810-bb6a-3946-3c0b83f7fe44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
game_library2.to_csv("games.csv")

### get genres

In [53]:
# function that puts response list object into a dataframe for each page
def get_genres(wrapper):
    genres = pd.DataFrame()
    for i in range (0, 409):
        genre = wrapper.api_request('genres', 'fields *; limit 500;')
        y = json.loads(genre)
        results_df =pd.DataFrame(y)
        genres = pd.concat([genres, results_df])
    return genres

In [ ]:
# function that puts response list object into a dataframe for each page
def get_game_library(wrapper):
    game_library = pd.DataFrame()
    for i in range (0, 409):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(games)
        results_df =pd.DataFrame(y)
        game_library = pd.concat([game_library, results_df])
    return game_library

In [54]:
genres = get_genres(wrapper)
genres

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b
...,...,...,...,...,...,...,...
18,34,1571788800,Visual Novel,visual-novel,1571788800,https://www.igdb.com/genres/visual-novel,137fe5c7-e4bc-1c30-4c7c-54596b47448f
19,32,1341360000,Indie,indie,1341360000,https://www.igdb.com/genres/indie,2522259f-2454-ec02-5dc0-84373e0508ed
20,35,1588809600,Card & Board Game,card-and-board-game,1588809600,https://www.igdb.com/genres/card-and-board-game,137bcfbe-be08-0b36-b66e-1bfef14ca49a
21,36,1588809600,MOBA,moba,1588809600,https://www.igdb.com/genres/moba,0f17b3a1-6d97-4c02-0997-850adf4dc624


In [21]:
genres.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

***Takeaways**

- drop  'created_at', 'updated_at', 'url', 'checksum'

- keep 'id', 'name', 'slug'

### get age ratings

In [12]:
def get_age_ratings(wrapper):
    age_ratings = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('age_ratings', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        age_ratings = pd.concat([age_ratings, results_df])
    return age_ratings

In [13]:
age_ratings = get_age_ratings(wrapper)
age_ratings

,id,category,rating,checksum,content_descriptions,synopsis
0,22474,1,8,361fd414-94d8-d6f8-6641-47abd3e8a75c,NaN,NaN
1,43430,1,8,5970434e-a37f-d43d-8975-2b23ab3370b3,NaN,NaN
2,2853,1,10,2d21809b-c282-95ba-69fb-5eda4d684b43,"[4244, 4245, 4246, 4247]",NaN
3,32550,1,9,40557417-e2a9-9a4f-5810-4159463467a1,[32972],NaN
4,65037,3,15,9c0f7e9a-8ceb-27f3-3514-cc6407a176cd,"[66283, 66284, 66285]",NaN
...,...,...,...,...,...,...
122,93004,3,15,467bd8c4-d8a1-23b1-8ff5-8723e997f7fc,[96991],NaN
123,93009,3,17,658ddc53-478b-2a23-75bc-e8d84658aa3c,[96997],NaN
124,93011,3,13,ccb03c90-860a-ce19-c395-8e4c2428800c,NaN,NaN
125,93010,3,14,bd576465-68d9-170d-6041-94455d8c8c59,"[96998, 96999, 97000]",NaN


In [26]:
age_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69266 entries, 0 to 265
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    69266 non-null  int64 
 1   category              69266 non-null  int64 
 2   rating                69266 non-null  int64 
 3   checksum              69266 non-null  object
 4   content_descriptions  34662 non-null  object
 5   synopsis              7062 non-null   object
dtypes: int64(3), object(3)
memory usage: 3.7+ MB


In [24]:
age_ratings.columns

Index(['id', 'category', 'rating', 'checksum', 'content_descriptions', 'synopsis'], dtype='object')

**TakeAways**

- delete  'checksum', 'content_descriptions', 'synopsis'
- keep id', 'category', 'rating'

### get age rating contents

In [62]:
def get_age_rating_desc(wrapper):
    age_ratings_desc = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('age_rating_content_descriptions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        age_ratings_desc = pd.concat([age_ratings_desc, results_df])
    return age_ratings_desc

In [63]:
age_rating_desc = get_age_rating_desc(wrapper)
age_rating_desc

,id,category,description,checksum
0,47,21.0,Strong Language,befd2777-5035-29b9-a592-3e0ba572b3bb
1,20,21.0,Strong Language,a0f5a6b5-3f93-00bc-f733-c5480af42132
2,50,21.0,Strong Language,21d63306-18a3-73e9-1a02-922431d31b0f
3,35,3.0,Blood,18821468-e01a-5629-67f8-36539a6cc69a
4,39,21.0,Strong Language,24807756-487a-29aa-8858-7781e3ce5e06
...,...,...,...,...
55,97124,68.0,Sexuality,19f86aca-7716-c00c-cc88-df40066dde0b
56,97122,68.0,Sexuality,19f86aca-7716-c00c-cc88-df40066dde0b
57,97123,69.0,Violence,950c5977-052d-1889-eb85-ddd363f82fa5
58,97125,69.0,Violence,950c5977-052d-1889-eb85-ddd363f82fa5


In [56]:
age_rating_desc.columns

Index(['id', 'category', 'description', 'checksum'], dtype='object')

**Takeaways**

- delete checksum 

### get characters

In [76]:
access_token = connect_api()

In [38]:
def get_characters(wrapper):
    characters = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('characters', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        characters = pd.concat([characters, results_df])
    return characters

In [39]:
characters = get_characters(wrapper)
characters

,id,created_at,games,name,slug,updated_at,url,checksum,gender,mug_shot,species,description,akas
0,13116,1555977600,[103020],Nuclear Launch Silo,nuclear-launch-silo,1555977600,https://www.igdb.com/characters/nuclear-launch...,bb6801a9-2b8f-6843-1964-2f440f1b8e48,NaN,NaN,NaN,NaN,NaN
1,14447,1562198400,[75242],Agent Carson,agent-carson,1562198400,https://www.igdb.com/characters/agent-carson,ac16307c-58d1-37cd-cda7-490f9345bb7b,NaN,NaN,NaN,NaN,NaN
2,504,1397865600,"[2765, 2911]",Gary,gary,1501632000,https://www.igdb.com/characters/gary,56dfd9a7-5cfa-ee91-14e5-46a493843313,NaN,NaN,NaN,NaN,NaN
3,2692,1414800000,[41],O,o,1458518400,https://www.igdb.com/characters/o,e32722d2-9798-5a72-b240-fdcbe43fabc5,NaN,NaN,NaN,NaN,NaN
4,6715,1449878400,[1958],Thophile Paddington,thophile-paddington,1449878400,https://www.igdb.com/characters/thophile-paddi...,51e57c03-5b7e-fc22-c838-1f7dfe753e92,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,16575,1647897710,[2369],Menelag,menelag,1647897729,https://www.igdb.com/characters/menelag,ecc219c8-c15f-7ee2-4bb5-8287ff53b418,NaN,NaN,NaN,NaN,NaN
392,16579,1647897711,[2369],Additional,additional,1647897732,https://www.igdb.com/characters/additional,075e65d8-c756-a0c9-9224-086ccfd9cfe3,NaN,NaN,NaN,NaN,NaN
393,4629,1432043872,[897],Cate Archer,cate-archer,1651237111,https://www.igdb.com/characters/cate-archer,1dbdc5b9-346d-5870-20c7-3627a132292e,1.0,4068.0,1.0,NaN,"[Catherine Ann Archer, The Fox]"
394,10236,1475914520,NaN,Funtime Freddy,funtime-freddy,1657731347,https://www.igdb.com/characters/funtime-freddy,3b31ca09-177a-0c37-eb02-06f979aa5f96,0.0,4069.0,4.0,Funtime Freddy is a major antagonist who first...,NaN


In [30]:
characters.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum', 'gender', 'mug_shot', 'species', 'description', 'akas'], dtype='object')

**Takeaways**

- delete  'created_at',  'updated_at', 'url', 'checksum', 'gender', 'mug_shot', 'species', 'description', 'akas'

- keep 'id', 'games', 'name', 'slug'

### get collections

In [83]:
def get_collections(wrapper):
    collections = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('collections', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        collections = pd.concat([collections, results_df])
    return collections

In [84]:
collections = get_collections(wrapper)
collections

,id,created_at,games,name,slug,updated_at,url,checksum
0,8202,1661995850,[189115],Grey Heritage,grey-heritage,1662330684,https://www.igdb.com/collections/grey-heritage,2e258dc4-d204-b8a8-df3a-29451234bc6b
1,7852,1651653419,[109972],Elsword,elsword,1651653420,https://www.igdb.com/collections/elsword,fe3864e8-5e0c-f27c-07cd-5ce3e4b3ffcf
2,4117,1507934083,"[11933, 18423, 39908, 61311, 61312, 61314]",Sprint,sprint,1507934083,https://www.igdb.com/collections/sprint,d0e1c01e-5c30-6ddb-2969-033fb82966f5
3,3247,1507276944,"[59261, 59263, 59264]",Hitozuma Unyu Saimin NTR Ben,hitozuma-unyu-saimin-ntr-ben,1507276944,https://www.igdb.com/collections/hitozuma-unyu...,fca09763-036f-9276-a361-2bb1b02365eb
4,5297,1516127813,"[77413, 77414, 77415, 77416, 77417]",LavaCat,lavacat,1516127813,https://www.igdb.com/collections/lavacat,34ee825d-4905-8420-5e0b-6400fe5c4826
...,...,...,...,...,...,...,...,...
318,3061,1506587780,"[67197, 67198, 68312, 68313, 68314, 68332, 683...",Arcade Archives,arcade-archives,1663676192,https://www.igdb.com/collections/arcade-archives,d9c0b1c2-cbc1-24ee-109d-8cfdea924c09
319,8281,1663322592,[207041],Hitsuji Shitsuji,hitsuji-shitsuji,1663329271,https://www.igdb.com/collections/hitsuji-shitsuji,136fdf3b-577d-bb95-963f-ac9346ba92a2
320,206,1321478702,"[843, 981, 1963, 1964, 1966, 1967, 1968, 2925,...",Rayman,rayman,1663754450,https://www.igdb.com/collections/rayman,9e989bcc-c35e-2491-03f8-a193f092ee15
321,39,1300365297,"[359, 385, 386, 387, 388, 392, 393, 395, 397, ...",Final Fantasy,final-fantasy,1663689053,https://www.igdb.com/collections/final-fantasy,ea3fd74e-18d0-dc93-c892-de2dd2934133


In [34]:
collections.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**
- delete 'created_at', 'updated_at', 'url', 'checksum'
- keep 'id', 'games', 'name', 'slug'

### get franchises

In [140]:
def get_franchises(wrapper):
    franchises = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('franchises', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        franchises = pd.concat([franchises, results_df])
    return franchises

In [141]:
franchises = get_franchises(wrapper)
franchises

,id,created_at,games,name,slug,updated_at,url,checksum
0,545,1428781176,"[6805, 9626, 10091, 11754, 11755, 11767, 11768...",VOCALOID,vocaloid,1428781176,https://www.igdb.com/franchises/vocaloid,d7a62e0a-1662-f2e4-1dde-1fc10c829e39
1,2574,1617402728,[145390],NextRev,nextrev,1617402728,https://www.igdb.com/franchises/nextrev,ac73431c-640d-8b37-c757-09c2135e1167
2,2798,1623025704,[151890],Tuned Turtle,tuned-turtle,1623025704,https://www.igdb.com/franchises/tuned-turtle,3220ee67-be4b-ce96-dd2e-241cfa976809
3,783,1458155907,[18385],Austin Powers,austin-powers,1458155907,https://www.igdb.com/franchises/austin-powers,c2e2e449-0b19-7bc3-9ed2-ceba6a5844aa
4,730,1446474129,NaN,Kane & Lynch,kane-and-lynch,1446474129,https://www.igdb.com/franchises/kane-and-lynch,81914463-391d-da66-9f96-3cd8fb085c2c
...,...,...,...,...,...,...,...,...
200,4006,1662655852703,NaN,Joey and Penguin,joey-and-penguin,1662655852,https://www.igdb.com/franchises/joey-and-penguin,cc39cbfe-697b-6209-3008-113ac97833be
201,51,1372173591416,"[138, 189, 190, 390, 422, 542, 2050, 2157, 217...",LEGO,lego,1663230468,https://www.igdb.com/franchises/lego,73dbe2df-dc2e-2be9-d6cd-144dff577638
202,4032,1663324223545,NaN,LEX Studios,lex-studios,1663324223,https://www.igdb.com/franchises/lex-studios,7bac82ac-f258-f502-cc8f-1e5fd77a7079
203,4043,1663422306558,NaN,Enmity,enmity,1663422306,https://www.igdb.com/franchises/enmity,73900f48-0c47-db35-0508-002b46a13a32


In [44]:
franchises.columns

Index(['id', 'created_at', 'games', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'updated_at', 'url', 'checksum'
- keep 'id', 'games', 'name', 'slug'

### get game engines

In [49]:
def get_game_engines(wrapper):
    game_engines = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_engines', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_engines = pd.concat([game_engines, results_df])
    return game_engines

In [48]:
game_engines = pd.DataFrame()

In [50]:
game_engines = get_game_engines(wrapper)
game_engines

,id,companies,created_at,logo,name,platforms,slug,updated_at,url,checksum
0,186,[5025],1425427200,31.0,OEngine,"[6, 48, 49]",oengine,1425427200,https://www.igdb.com/game_engines/oengine,7b4118c7-5081-d1de-8fa7-c122074254da
1,1410,[14081],1649979323,NaN,AI5,"[13, 118, 121, 149]",ai5,1652857462,https://www.igdb.com/game_engines/ai5,6bb11504-0a8e-67e1-ff46-46c70fc177bd
2,1291,NaN,1627144462,NaN,Pinball Construction Set,NaN,pinball-construction-set,1627506462,https://www.igdb.com/game_engines/pinball-cons...,2b15d014-4f7e-fcb0-addf-6c5c9c653388
3,237,[356],1431907200,NaN,The Madness Engine,NaN,the-madness-engine,1578700800,https://www.igdb.com/game_engines/the-madness-...,bf7f5406-4568-4aaf-b2b5-7c4bc106e3a3
4,975,NaN,1590935612,NaN,TIC-80,NaN,tic-80,1643881103,https://www.igdb.com/game_engines/tic-80,5b658b0b-b4a1-82e4-78fc-189ab707eede
...,...,...,...,...,...,...,...,...,...,...
129,820,NaN,1577779095,NaN,LithTech Jupiter EX,NaN,lithtech-jupiter-ex,1650853805,https://www.igdb.com/game_engines/lithtech-jup...,b7ff44d5-3226-0e2f-11ca-3c49935aa113
130,79,[6932],1411945309,0.0,GameMaker: Studio,"[3, 6, 9, 14, 34, 39, 46, 48, 49, 74]",gamemaker-studio,1663572164,https://www.igdb.com/game_engines/gamemaker-st...,58d6cd85-b8be-f581-67c9-72f2abcee2fd
131,626,NaN,1553127899,NaN,LithTech Jupiter,NaN,lithtech-jupiter,1650856108,https://www.igdb.com/game_engines/lithtech-jup...,6c6d9db1-43c3-0d79-e481-56317a7ae281
132,296,[7466],1452509450,43.0,Clausewitz Engine,NaN,clausewitz-engine,1651025009,https://www.igdb.com/game_engines/clausewitz-e...,b6e16986-c822-60e8-b70d-f26b3031764a


In [52]:
game_engines.columns

Index(['id', 'companies', 'created_at', 'logo', 'name', 'platforms', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'logo',  'updated_at', 'url', 'checksum'

### endpoints

In [ ]:
endpoints = [
    'https://api.igdb.com/v4/age_ratings',
    'https://api.igdb.com/v4/age_rating_content_descriptions',
    'https://api.igdb.com/v4/characters',
    'https://api.igdb.com/v4/companies',
    'https://api.igdb.com/v4/collections',
    'https://api.igdb.com/v4/franchises',
    'https://api.igdb.com/v4/game_engines'
    ]

### get game modes

In [97]:
def get_game_modes(wrapper):
    game_modes = pd.DataFrame()
    for i in range (0, 1):
        a_ratings = wrapper.api_request('game_modes', 'fields *; limit 500;')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_modes = pd.concat([game_modes, results_df])
    return game_modes

In [98]:
game_modes = get_game_modes(wrapper)
game_modes

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,Single player,single-player,1323216000,https://www.igdb.com/game_modes/single-player,a43b1688-f968-3541-0897-9735ffde1745
1,2,1298937600,Multiplayer,multiplayer,1323216000,https://www.igdb.com/game_modes/multiplayer,288b548c-11e4-d910-f037-21d4e6a33b38
2,3,1298937600,Co-operative,co-operative,1323216000,https://www.igdb.com/game_modes/co-operative,e8329d55-33e6-f7ae-ae09-2848cb7ccc90
3,4,1298937600,Split screen,split-screen,1323216000,https://www.igdb.com/game_modes/split-screen,98247b75-6f46-7c77-521f-0945c684d842
4,5,1298937600,Massively Multiplayer Online (MMO),massively-multiplayer-online-mmo,1323216000,https://www.igdb.com/game_modes/massively-mult...,307d1126-6e3b-9215-06f1-10c8ecce05b4
5,6,1588809600,Battle Royale,battle-royale,1588809600,https://www.igdb.com/game_modes/battle-royale,bfc388ac-0502-ba05-af25-3dcc94a0acf3


In [81]:
game_modes.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

- delete  'created_at', 'updated_at', 'url', 'checksum'

### get game version

In [88]:
def get_game_versions(wrapper):
    game_versions = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_versions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_versions = pd.concat([game_versions, results_df])
    return game_versions

In [84]:
game_versions = get_game_versions(wrapper)
game_versions

,id,created_at,game,updated_at,url,checksum,features,games
0,241,1596585600,26192,1596585600,https://www.igdb.com/game_versions/the-last-of...,e71699ef-97ce-af28-c6b1-a5272393fcfa,NaN,NaN
1,8,1505174400,31311,1505174400,https://www.igdb.com/game_versions/monsti/,4db869ec-e1a3-3f1d-8d88-04d5609d21db,NaN,NaN
2,7,1505174400,20668,1505174400,https://www.igdb.com/game_versions/zombie-driver/,f2b2058d-7c6a-a793-5a88-bfc63871005b,NaN,NaN
3,13,1505174400,52983,1505174400,https://www.igdb.com/game_versions/eternal-jou...,9f694e8f-b4f7-2e97-479a-5190b328bd0d,NaN,NaN
4,10,1505174400,32721,1505174400,https://www.igdb.com/game_versions/zion/,55f1ff2a-7605-4bfc-da2b-ac7781971b5f,NaN,NaN
...,...,...,...,...,...,...,...,...
186,206,1573171200,13664,1573171200,https://www.igdb.com/game_versions/distraint/,6a1f626a-879d-d1bd-2390-3b6e4b2f9f0b,NaN,NaN
187,44,1505952000,7349,1505952000,https://www.igdb.com/game_versions/mass-effect...,d749f321-1e08-5674-438a-13df9e354ada,"[99, 100, 101, 102, 103]","[7349, 27766, 27767]"
188,45,1506038400,27421,1535500800,https://www.igdb.com/game_versions/middle-eart...,eca61d61-f746-ad65-915f-5e0fe7637738,"[253, 254, 255, 256, 257, 258, 259, 260, 261, ...","[27421, 36986, 53358, 53359, 106764]"
189,1,1505174400,28540,1507420800,https://www.igdb.com/game_versions/assassins-c...,4ff85042-d876-4aff-5589-25167ce699a7,"[50, 51, 52, 98, 135, 136, 137, 138, 139, 140,...","[28540, 39034, 39044, 39045, 39046, 39047]"


In [85]:
game_versions.columns

Index(['id', 'created_at', 'game', 'updated_at', 'url', 'checksum', 'features', 'games'], dtype='object')

**Takeawys**

- delete  'created_at', 'updated_at', 'url', 'checksum', 

### get game version features

In [89]:
def get_game_version_features(wrapper):
    game_version_features = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_features', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_features = pd.concat([game_version_features, results_df])
    return game_version_features

In [90]:
game_version_features = get_game_version_features(wrapper)
game_version_features

,id,category,description,position,title,values,checksum
0,337,0.0,Sharpens your aim with rifles and pistols. Alw...,1,DLC: The Sharpshooter Detective Suit,"[1385, 1386, 1387, 1388]",a10d3ebf-ae3a-f0cc-fc68-116b1545a9ac
1,347,0.0,NaN,0,"25 cm (10"") figurine of an Alb Mage","[1419, 1420]",2a42989b-4936-75bc-6331-0ce79abb225a
2,352,0.0,NaN,5,A making of DVD,"[1429, 1430]",bb46d55f-7145-0b5c-2f69-cea9dd2570d6
3,375,0.0,Double sided print of the in-game island,2,Collectible map,"[1519, 1520, 1521, 1522, 1523]",8a9ea6d0-b5f1-ece2-cd5d-61636d639ad0
4,382,0.0,"Exclusive to TESCO in the UK, it included thre...",9,Combat Strike Pack,"[1554, 1555, 1556, 1557, 1558]",263031ce-acb2-69a2-ab66-240dbaad4f4d
...,...,...,...,...,...,...,...
440,17,NaN,NaN,0,Season Pass,NaN,c27cf85d-4af5-9fc1-6f59-52a9624f9490
441,456,0.0,created in collaboration with Dark Horse Comic...,1,Full-color print comic book,"[1773, 1774, 1775, 1776]",218da9c1-cc23-cc6d-a50a-72ba78203f66
442,459,0.0,Full-color printed map of the world of Conan E...,4,World Map,"[1785, 1786, 1787, 1788]",3f282aa8-abeb-8fb8-2dc7-5277c451c0aa
443,555,0.0,NaN,0,Limited edition figurine,"[2154, 2155]",a655f38f-4322-fb48-6d4e-8bc71f3834c0


In [91]:
game_version_features.columns

Index(['id', 'category', 'description', 'position', 'title', 'values', 'checksum'], dtype='object')

**Takeaways**

-  delete  'checksum'

### get version feature values

In [ ]:
def get_version_features_values():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; limit 500;'
    r = requests.post('https://api.igdb.com/v4/game_version_feature_values' ,data=data ,  headers=headers)
    game_version_feature_values = wrapper.api_request('game_version_feature_values', 'fields *;limit 500;')
    x = json.loads(game_version_feature_values)
    game_versions_feature_values =pd.DataFrame(x)
    return game_versions_feature_values

In [92]:
def get_game_version_feature_values(wrapper):
    game_version_feature_values = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_feature_values', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_feature_values = pd.concat([game_version_feature_values, results_df])
    return game_version_feature_values

In [93]:
game_version_feature_values = get_game_version_feature_values(wrapper)
game_version_feature_values

,id,game,game_feature,included_feature,checksum,note
0,1581,52617,391,0.0,1db62035-9135-3d01-25a5-9c194d8fbd8f,NaN
1,1590,52617,394,0.0,e2e097d5-c88d-d6ea-48cf-1f5288f6bbe2,NaN
2,591,39045,144,0.0,6ee07864-0f1b-d41f-e2a5-e7dc2731d9a0,NaN
3,1987,83886,503,2.0,612d4e1b-a3b2-60e4-680a-064c0837f71e,NaN
4,1600,19333,399,0.0,ba6dcc29-45c7-6412-ebce-da6731910fb9,NaN
...,...,...,...,...,...,...
17,1983,53012,503,2.0,74eacd56-d0fa-62ab-a961-df386c4cd140,NaN
18,2155,193391,555,1.0,2eb547da-7c57-147e-1bdd-9d9b8dcc3eac,NaN
19,2154,5116,555,0.0,d257f647-5899-a6e0-fca6-516809965ba1,NaN
20,2156,5116,556,0.0,bc29ab0a-9945-f712-2de9-752bb23c3524,NaN


In [97]:
game_version_feature_values.columns

Index(['id', 'game', 'game_feature', 'included_feature', 'checksum', 'note'], dtype='object')

Takeaways

- delete 'checksum' 'note'

### get companies

In [98]:
def get_companies(wrapper):
    companies= pd.DataFrame()
    for i in range (0, 409):
        company = wrapper.api_request('companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(company)
        results_df =pd.DataFrame(y)
        companies = pd.concat([companies, results_df])
    return companies

In [99]:
companies = get_companies(wrapper)
companies

,id,change_date_category,country,created_at,description,developed,logo,name,slug,start_date,start_date_category,updated_at,url,websites,checksum,published,change_date,changed_company_id,parent
0,13710,7,752.0,1508976000,Tomolo Games develope fun and simple mobile ga...,"[74974, 74975, 74977]",1396.0,Tomolo Games,tomolo-games,1.494634e+09,0,1508976000,https://www.igdb.com/companies/tomolo-games,[1089],656407de-863f-13bc-82c0-16079a13c964,NaN,NaN,NaN,NaN
1,15094,7,233.0,1520899200,NaN,[26472],NaN,ZA/UM,za-slash-um,NaN,7,1573948800,https://www.igdb.com/companies/za-slash-um,"[1944, 3431]",d32b01ce-2272-3bf2-e809-1f16cf5c86d9,[26472],NaN,NaN,NaN
2,17841,7,246.0,1548720000,NaN,[114715],NaN,Moido Games,moido-games,NaN,7,1548720000,https://www.igdb.com/companies/moido-games,[1683],e48ef63d-fdc0-35a2-bbc6-db61277e2c25,NaN,NaN,NaN,NaN
3,16305,7,208.0,1535673600,"PortaPlay is an independent developer, residin...",[107175],1748.0,PortaPlay,portaplay,1.181952e+09,0,1536019200,https://www.igdb.com/companies/portaplay,[1323],5d28752a-62f8-f617-f0d5-6ac59c0efa6e,NaN,NaN,NaN,NaN
4,4270,7,392.0,1418428800,T's Music (ティーズ ミュージック Tiizu Myuujikku) is a g...,NaN,3304.0,T's Music,ts-music,-6.213568e+10,2,1576627200,https://www.igdb.com/companies/ts-music,[3517],238c892a-7f60-78f2-415a-7f114ccda898,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,43205,7,NaN,1663132230,Wheat is a Aspiring game dev a TF2Classic deve...,[217612],6693.0,Wheat,wheat,NaN,7,1663137702,https://www.igdb.com/companies/wheat,[6266],ddba68cf-1bcc-db60-1ce7-a23858656ca7,NaN,NaN,NaN,NaN
496,43207,7,578.0,1663153197,Hello Bard A/S is an independent game developm...,[37019],NaN,Hello Bard,hello-bard,1.483142e+09,2,1663154687,https://www.igdb.com/companies/hello-bard,[6267],58cb7354-fa27-a025-ed0d-0e7dae37e356,[37019],NaN,NaN,NaN
497,34124,7,250.0,1621274945,NaN,NaN,6696.0,Nobilis Interactive,nobilis-interactive,1.199059e+09,2,1663154776,https://www.igdb.com/companies/nobilis-interac...,NaN,8a1256de-6ebd-5240-bbba-bb5782a29930,[148055],NaN,NaN,NaN
498,43209,7,40.0,1663171356,Graphic design & Animation Company,NaN,6698.0,JoonyTv,joonytv,1.647994e+09,0,1663179583,https://www.igdb.com/companies/joonytv,[6269],1a53e5a1-6488-09df-36a2-83decc22bb8a,NaN,NaN,NaN,NaN


In [100]:
companies.columns

Index(['id', 'change_date_category', 'country', 'created_at', 'description', 'developed', 'logo', 'name', 'slug', 'start_date', 'start_date_category', 'updated_at', 'url', 'websites', 'checksum', 'published', 'change_date', 'changed_company_id', 'parent'], dtype='object')

**Takeaways**

- delete 'id', 'change_date_category', 'country', 'created_at', 'description', 'developed', 'logo', 'name', 'slug', 'start_date', 'start_date_category', 'updated_at', 'url', 'websites', 'checksum', 'published', 'change_date', 'changed_company_id', 'parent'

### get involved companies

In [159]:
def get_involved_companies(wrapper):
    involved_companies= pd.DataFrame()
    for i in range (0, 409):
        i_company = wrapper.api_request('involved_companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(i_company)
        results_df =pd.DataFrame(y)
        involved_companies = pd.concat([involved_companies, results_df])
    return involved_companies

In [161]:
involved_companies = get_involved_companies(wrapper)
involved_companies

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
0,95000,14486,1585267200,True,131992,False,False,False,1585440000,43e91be1-592f-c3be-3e1d-447b3f17584c
1,78799,1250,1557532800,True,20689,False,False,False,1557532800,169b9e17-f2e5-ba93-4b58-97ec89978c68
2,20543,19,1419033600,True,602,False,True,False,1419033600,3058ba74-b6d7-f755-d603-1ff10337c15f
3,36564,10142,1466121600,True,19671,False,False,False,1466294400,d88519f6-e9bb-fd6b-40a9-c58ccad78312
4,66470,2888,1531440000,True,105048,False,False,False,1531440000,77bc32cf-a18b-ef18-8099-ed3edb7e4ca2
...,...,...,...,...,...,...,...,...,...,...
246,178623,112,1657711901,False,45730,False,True,False,1657714382,5f448d4d-9be8-2bb0-0f2c-d93343c70919
247,178635,1274,1657714406,False,49977,False,True,False,1657714419,52e2b7d6-3a1f-9854-1499-d759591add28
248,178639,1274,1657715287,False,91775,False,True,False,1657715331,99ff7e26-b656-8cb2-15fd-87fe4595c690
249,185668,2064,1662084802,False,215085,False,True,False,1662085372,2079637d-afe6-22be-9c2c-d34961c93f80


In [162]:
involved_companies.columns

Index(['id', 'company', 'created_at', 'developer', 'game', 'porting', 'publisher', 'supporting', 'updated_at', 'checksum'], dtype='object')

**Takeaways**

### get multi player modes

In [126]:
def get_multi_player_modes(wrapper):
    multi_player_modes= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('multiplayer_modes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        multi_player_modes = pd.concat([multi_player_modes, results_df])
    return multi_player_modes

In [127]:
multi_player_modes = get_multi_player_modes(wrapper)
multi_player_modes

,id,campaigncoop,dropin,game,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum,offlinecoopmax,onlinecoopmax,onlinemax
0,9953,False,False,92273,False,False,2.0,False,22.0,False,1b94d937-dc0c-c62d-10af-e58d42860bd8,NaN,NaN,NaN
1,1832,True,True,7153,False,True,0.0,False,12.0,False,2b09f64f-b8b9-6948-ad98-9ab9bede8e3b,2.0,0.0,0.0
2,7987,False,False,57887,False,True,2.0,False,49.0,False,56c9ee4c-aae3-ba3c-ead0-9e899e1c5267,0.0,0.0,0.0
3,7,False,False,46076,False,False,30.0,False,6.0,False,a41bcf9c-976d-8690-629f-4c2737f65bf1,NaN,NaN,NaN
4,10207,False,False,31256,False,False,NaN,False,82.0,False,788009fd-9e08-5826-577b-9844575ba2d9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,20775,False,False,218204,False,False,NaN,False,6.0,False,2836d768-73f2-3700-840e-a63d86599f52,NaN,NaN,NaN
245,20774,False,False,218201,False,False,NaN,False,6.0,False,dc98c1d7-c02b-67af-44ca-c59d9d1c5a98,NaN,NaN,NaN
246,20782,False,False,201498,False,False,NaN,True,6.0,False,3d75a41b-69ba-4f72-1cf0-84ffa111092a,NaN,4.0,NaN
247,20784,False,True,218300,False,False,1.0,True,6.0,False,0066f411-3aed-d2db-7a62-214939a0b2fd,1.0,4.0,8.0


In [165]:
multi_player_modes.columns

Index(['id', 'campaigncoop', 'dropin', 'game', 'lancoop', 'offlinecoop', 'offlinemax', 'onlinecoop', 'platform', 'splitscreen', 'checksum', 'offlinecoopmax', 'onlinecoopmax', 'onlinemax'], dtype='object')

**Takeaways**

### get families

In [166]:
def get_platform_families(wrapper):
    platform_families= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_families', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_families = pd.concat([platform_families, results_df])
    return platform_families

In [167]:
platform_families = get_platform_families(wrapper)
platform_families

,id,name,slug,checksum
0,5,Nintendo,nintendo,398e5c87-f7db-6265-6da6-433441385208
1,4,Linux,linux,be8b89fa-823c-790a-1d5a-7e74315c1b27
2,2,Xbox,xbox,5bb9bcec-4aa7-45b8-129b-a6f88932942e
3,3,Sega,sega,bd1a9cc4-3632-33b0-ff4b-feab8cd15001
4,1,PlayStation,playstation,eadc0208-7972-9364-88b9-d2a9de75a998


In [168]:
platform_families.columns

Index(['id', 'name', 'slug', 'checksum'], dtype='object')

**Takeaways**

### get platform v release dates

In [172]:
def get_platform_version_releases(wrapper):
    platform_version_release_dates= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_version_release_dates', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_version_release_dates = pd.concat([platform_version_release_dates, results_df])
    return platform_version_release_dates

In [174]:
platform_version_release_dates = get_platform_version_releases(wrapper)
platform_version_release_dates

,id,category,date,human,m,region,y,checksum,updated_at
0,555,2,567907200,1987,12,8,1987,186c07ec-0d93-dc0f-6c4a-5fdb5a6138fa,NaN
1,338,0,1189555200,"Sep 12, 2007",9,3,2007,15e5b0aa-5bb6-ef30-ee85-0ef94cb3581f,NaN
2,334,0,1125532800,"Sep 01, 2005",9,7,2005,a4c921d5-1894-9a40-d20f-05959cd69a38,NaN
3,331,0,1415923200,"Nov 14, 2014",11,3,2014,3ebb8dc8-b8fa-f11a-b729-228b9dac35c4,NaN
4,333,0,1389830400,"Jan 16, 2014",1,7,2014,a15ecea5-cb71-916f-5bec-66ad7f571b19,NaN
...,...,...,...,...,...,...,...,...,...
391,660,0,1547942400,"Jan 20, 2019",1,7,2019,29210c4c-26c6-212c-e0ec-c0d6bdd23be9,1.660316e+09
392,659,0,1413763200,"Oct 20, 2014",10,2,2014,ddac84c3-8f5a-8372-fee3-5e05cc9517f5,1.660316e+09
393,661,2,725760000,1992,12,8,1992,d6cb796b-0679-d99b-30d8-decdf573187d,1.660320e+09
394,662,0,945043200,"Dec 13, 1999",12,5,1999,6fa54af0-85f8-088f-294c-d95dbbbb2a70,1.660322e+09


In [175]:
platform_version_release_dates.columns

Index(['id', 'category', 'date', 'human', 'm', 'region', 'y', 'checksum', 'updated_at'], dtype='object')

**Takeaways**

### get platform version companies

In [177]:
def get_platform_version_companies(wrapper):
    platform_version_release_companies= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_version_companies', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_version_release_companies = pd.concat([platform_version_release_companies, results_df])
    return platform_version_release_companies

In [180]:
platform_version_release_companies = get_platform_version_companies(wrapper)

platform_version_release_companies


,id,company,developer,manufacturer,checksum,comment
0,286,11986,True,True,e5035f31-713a-1036-2968-dd353b74db3b,NaN
1,283,10922,False,True,6cd3e57c-2757-cc07-94a6-35ff7085e4c9,NaN
2,282,14429,False,True,172b6f24-a49c-c304-6fb3-78f6bf120a19,NaN
3,281,12385,True,False,d6051460-e7dc-4e77-80ca-e5df378cd304,NaN
4,280,19831,True,True,2f509e1b-75d2-36db-4129-4bf89d50343b,NaN
...,...,...,...,...,...,...
424,650,40695,True,True,34204aec-5719-bc70-eaa9-80d649d4acac,NaN
425,651,70,True,False,c5b2ca2d-92e3-0d15-c40c-66603c6773fe,NaN
426,652,42680,True,True,aaf4cd13-c03f-83ae-4e84-2d9d98fad050,NaN
427,653,42681,True,False,4b080df5-0b43-df63-7d2d-938a3dcbd8a9,NaN


In [181]:
platform_version_release_companies.columns

Index(['id', 'company', 'developer', 'manufacturer', 'checksum', 'comment'], dtype='object')

**Takeaways**

### get platforms

In [113]:
def get_platforms(wrapper):
    platforms= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platforms', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platforms = pd.concat([platforms, results_df])
    return platforms

In [114]:
platforms = get_platforms(wrapper)
platforms

,id,alternative_name,category,created_at,name,platform_logo,slug,updated_at,url,versions,websites,checksum,generation,platform_family,abbreviation,summary
0,158,Commodore Dynamic Total Vision,6.0,1510012800,Commodore CDTV,292.0,commodore-cdtv,1522972800,https://www.igdb.com/platforms/commodore-cdtv,[223],[26],5d48648e-283c-e2df-df7d-6d3cbc7e0a58,NaN,NaN,NaN,NaN
1,339,Kids Computer Pico,1.0,1595808000,Sega Pico,NaN,sega-pico,1595808000,https://www.igdb.com/platforms/sega-pico,[456],NaN,6e867a5b-a071-fc7b-945b-a0622f64a4bc,4.0,3.0,NaN,NaN
2,8,PS2,1.0,1297555200,PlayStation 2,254.0,ps2,1414972800,https://www.igdb.com/platforms/ps2,"[58, 114]",NaN,be3e025c-d4ed-b795-443c-a8cb55fbfe17,6.0,1.0,PS2,NaN
3,39,NaN,4.0,1317686400,iOS,248.0,ios,1391644800,https://www.igdb.com/platforms/ios,[43],NaN,fefe4a2b-7c90-0e89-e811-902ea1cf2b58,NaN,NaN,iOS,NaN
4,94,NaN,6.0,1414195200,Commodore Plus/4,310.0,c-plus-4,1468454400,https://www.igdb.com/platforms/c-plus-4,[108],NaN,ea138789-6bbe-bcca-dff5-7e20e762f65a,NaN,NaN,C+4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,415,NaN,5.0,1660319701,Watara/QuickShot Supervision,691.0,watara-slash-quickshot-supervision,1660326748,https://www.igdb.com/platforms/watara-slash-qu...,[555],[314],4ed0c0ee-3103-4d53-c0dc-612a391de28d,4.0,NaN,NaN,NaN
193,414,LeapTV VCD,1.0,1660314942,LeapTV,690.0,leaptv,1660326997,https://www.igdb.com/platforms/leaptv,[554],[313],74a35c87-865a-52ee-29f0-70c03039ff84,8.0,NaN,NaN,NaN
194,412,Leapster Learning Game System,5.0,1660246910,Leapster,689.0,leapster,1660327194,https://www.igdb.com/platforms/leapster,[552],[311],2a4b5882-ce1b-2ba6-f4c2-0f3e55fda0b9,6.0,NaN,NaN,NaN
195,438,NaN,5.0,1663685343,Arduboy,NaN,arduboy,1663685354,https://www.igdb.com/platforms/arduboy,[588],[337],54e5a3b8-49fa-e0b3-c5e7-b7dce5ef22ba,8.0,NaN,NaN,NaN


In [183]:
platforms.columns

Index(['id', 'alternative_name', 'category', 'created_at', 'name', 'platform_logo', 'slug', 'updated_at', 'url', 'versions', 'websites', 'checksum', 'generation', 'platform_family', 'abbreviation', 'summary'], dtype='object')

**Takeaways**

### get release dates

In [184]:
def get_release_dates(wrapper):
    release_dates= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('release_dates', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        release_dates = pd.concat([release_dates, results_df])
    return release_dates

In [185]:
release_dates = get_release_dates(wrapper)
release_dates

,id,category,created_at,date,game,human,m,platform,region,updated_at,y,checksum
0,170111,0,1558826090,1.552954e+09,76870,"Mar 19, 2019",3.0,48,8,1558826519,2019.0,eda4c3fc-5ce2-5c43-9a96-bfd20b24ab88
1,165447,0,1550867412,1.561421e+09,115477,"Jun 25, 2019",6.0,6,8,1550867489,2019.0,3f7a3186-1622-4da6-fb05-d223cea14454
2,85558,0,1497948084,5.660928e+08,38035,"Dec 10, 1987",12.0,121,5,1497948122,1987.0,df4b2c61-916b-d823-cdf7-286f06aeac62
3,26259,0,1422372749,1.079654e+09,2473,"Mar 19, 2004",3.0,24,1,1422377563,2004.0,b71a705c-751f-f832-18e5-1485482906db
4,310515,0,1631498978,6.317568e+08,48629,"Jan 08, 1990",1.0,99,5,1631499382,1990.0,7b655a08-c93d-d413-6ba7-8f5946acf34c
...,...,...,...,...,...,...,...,...,...,...,...,...
495,331150,7,1639701154,NaN,109582,TBD,NaN,6,8,1639701339,NaN,ff8bd362-af93-2abf-6120-7f06f62fc186
496,331153,7,1639701154,NaN,109582,TBD,NaN,14,8,1639701341,NaN,ca753dca-2830-d1c4-307d-98dc89b8e7f4
497,331155,0,1639701214,1.613606e+09,61467,"Feb 18, 2021",2.0,48,8,1639701338,2021.0,90ef6b66-8efa-b169-b905-eba7972c2285
498,331148,2,1639701093,1.640909e+09,28881,2021,12.0,161,8,1639701343,2021.0,3311a13c-476a-d350-2999-7d90462ed734


In [186]:
release_dates.columns

Index(['id', 'category', 'created_at', 'date', 'game', 'human', 'm', 'platform', 'region', 'updated_at', 'y', 'checksum'], dtype='object')

**Takeaways**

### get platform versions

In [187]:
def get_platform_versions(wrapper):
    platform_versions= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('platform_versions', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        platform_versions = pd.concat([platform_versions, results_df])
    return platform_versions

In [189]:
platform_versions = get_platform_versions(wrapper)
platform_versions

,id,companies,cpu,graphics,memory,name,os,platform_logo,platform_version_release_dates,slug,sound,storage,summary,url,checksum,media,resolutions,online,output,connectivity,main_manufacturer
0,103,[533],MOS Technology 6502 @ 1 MHz,monochrome character graphics,4–96 KB,Initial version,Commodore BASIC 1.0 ~ 4.0,543.0,[552],initial-version,none or beeper,"cassette tape, 5.25-inch floppy, 8-inch floppy...",The Commodore PET is a line of home/personal c...,https://www.igdb.com/platforms/cpet/version/in...,df0a5020-309d-04c9-5368-b2b9244bb6b4,NaN,NaN,NaN,NaN,NaN,NaN
1,19,NaN,Motorola 68000 @ 7.16 MHz (NTSC) 7.09 MHz (PAL),NaN,512 kB (9 MB maximum),Amiga A 500,AmigaOS v1.2 - 1.3,391.0,[134],amiga-a-500,4× 8-bit channels PCM at max 28 kHz with 6-bit...,NaN,"In 1982, the Amiga Corporation developed ""Lorr...",https://www.igdb.com/platforms/amiga/version/a...,8be75330-7847-49d4-7812-cae1b58d8e63,880 kB floppy disks,"736x567i 4-bpp PAL (736x483i 4 bpp NTSC), 368x...",NaN,NaN,NaN,NaN
2,282,[527],NaN,NaN,NaN,Switch Lite,NaN,534.0,[359],switch-lite,NaN,NaN,"A smaller version of the Switch, which only su...",https://www.igdb.com/platforms/switch/version/...,24097bf5-9bbf-5959-931b-4b87ece75fb8,NaN,NaN,NaN,NaN,NaN,NaN
3,139,[144],Ricoh 5A22 @ 3.58 MHz,NaN,128 KB,Super Famicom (SHVC-001),NaN,105.0,[204],super-famicom-shvc-001,"Nintendo S-SMP 8 channels, stereo, 64 KB",NaN,Original Japanese version (SHVC-001),https://www.igdb.com/platforms/snes--1/version...,3fcc6ae9-ac67-dc57-a433-780e25ee615f,ROM cartridge,256x224 to 512x448,"Satellaview (satellite radio), XBAND,","composite video, S-Video and RGB",NaN,NaN
4,164,NaN,NaN,NaN,NaN,Initial version,NaN,372.0,NaN,initial-version,NaN,NaN,NaN,https://www.igdb.com/platforms/trs-80/version/...,93cf119c-a0d5-5de9-fcaa-4692e0bd605c,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,553,[647],393 MHz ARM9-based processor,NaN,64 MB DDR SDRAM,Initial version,Linux,NaN,[658],initial-version,NaN,512 MB,The Leapster Explorer is a handheld console de...,https://www.igdb.com/platforms/leapster-explor...,ddd842c0-db98-ec71-1bae-ef855816426e,ROM Cartridge,320x240,NaN,NaN,NaN,NaN
308,554,"[648, 649]",1 GHz quad core custom processor,NaN,1GB DDR3 RAM,Initial version,NaN,690.0,"[659, 660]",initial-version,NaN,16 GB,LeapTV is an educational video game console de...,https://www.igdb.com/platforms/leaptv/version/...,0d18d1da-477d-77af-8331-f4c17ffd1afb,"ROM Cartridge, Video CD (Malaysia only)","1280x720, 640x480 (Malaysia only)",LeapFrog,"HDMI, composite video (Malaysia only)","Wi-Fi, Bluetooth",NaN
309,555,[650],4 MHz 8-bit 65SC02,NaN,NaN,Initial version,NaN,691.0,[661],initial-version,"2 Tonal + 1 Noise Channel, additional audio DM...",NaN,"The Watara Supervision, also known as the Quic...",https://www.igdb.com/platforms/watara-slash-qu...,52336677-f6d7-2067-f113-8cc6586b23bf,ROM Cartridge,160x160 in 4 color greyscale,NaN,TV Adpater,Two Player Link using DE-9 connector,NaN
310,556,"[651, 652]",NaN,NaN,NaN,Initial version,NaN,692.0,[662],initial-version,NaN,Rewritable magnetic disk,The Nintendo 64DD is a magnetic floppy disk dr...,https://www.igdb.com/platforms/nintendo-64dd/v...,401ff879-d203-c235-1c1a-ee8d5cac8c40,64 MB magnetic disk,NaN,Randnet,NaN,22.8 kbps dial-up modem,NaN


In [190]:
platform_versions.columns

Index(['id', 'companies', 'cpu', 'graphics', 'memory', 'name', 'os', 'platform_logo', 'platform_version_release_dates', 'slug', 'sound', 'storage', 'summary', 'url', 'checksum', 'media', 'resolutions', 'online', 'output', 'connectivity', 'main_manufacturer'], dtype='object')

**Takeaways**

### get player perspectives

In [106]:
def get_player_perspectives(wrapper):
    player_perspectives= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('player_perspectives', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        player_perspectives = pd.concat([player_perspectives, results_df])
    return player_perspectives

In [107]:
player_perspectives = get_player_perspectives(wrapper)
player_perspectives

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,First person,first-person,1323216000,https://www.igdb.com/player_perspectives/first...,4e23cb22-7a70-effb-b8e1-151317c6cdbd
1,2,1298937600,Third person,third-person,1323216000,https://www.igdb.com/player_perspectives/third...,2788b856-580c-66d0-bef3-d6169034f175
2,3,1298937600,Bird view / Isometric,bird-view-slash-isometric,1582502400,https://www.igdb.com/player_perspectives/bird-...,83c59132-6edd-150f-a25f-a86ddf6a0da3
3,5,1321228800,Text,text,1323216000,https://www.igdb.com/player_perspectives/text,e2df3906-bca8-e0d1-ca3b-67e1c5cfb334
4,4,1298937600,Side view,side-view,1323216000,https://www.igdb.com/player_perspectives/side-...,2d038ac6-36c5-af5b-83fd-11a88a638631
5,7,1462233600,Virtual Reality,virtual-reality,1462233600,https://www.igdb.com/player_perspectives/virtu...,963f8a54-d40f-1c9d-3bf4-d15267043693
6,6,1413158400,Auditory,auditory,1582502400,https://www.igdb.com/player_perspectives/auditory,74528cf1-64c3-cd78-18df-fae1594ee664


In [192]:
player_perspectives.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

### get themes

In [194]:
def get_themes(wrapper):
    themes= pd.DataFrame()
    for i in range (0, 409):
        modes = wrapper.api_request('themes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(modes)
        results_df =pd.DataFrame(y)
        themes = pd.concat([themes, results_df])
    return themes

In [195]:
themes = get_themes(wrapper)
themes

,id,created_at,name,slug,updated_at,url,checksum
0,20,1322524800,Thriller,thriller,1323216000,https://www.igdb.com/themes/thriller,4ebeaa43-4165-5d23-142a-9bb2a26c7024
1,18,1322524800,Science fiction,science-fiction,1323216000,https://www.igdb.com/themes/science-fiction,fffad274-fee4-e195-eb78-72b2a35d643e
2,1,1322524800,Action,action,1323216000,https://www.igdb.com/themes/action,c177e0ff-f29a-2a2e-fce5-f945258ceb59
3,19,1322524800,Horror,horror,1323216000,https://www.igdb.com/themes/horror,28762ccf-724f-56cf-3784-dbf16dfcf174
4,21,1322524800,Survival,survival,1323216000,https://www.igdb.com/themes/survival,01d5251e-746d-d9b6-6a47-99e8e9fda8ff
5,17,1322524800,Fantasy,fantasy,1323216000,https://www.igdb.com/themes/fantasy,454bdc25-3dbf-5648-1e11-fc67ffc893aa
6,22,1322524800,Historical,historical,1323216000,https://www.igdb.com/themes/historical,1256e1bd-db05-276a-16ad-9704f7a56dcc
7,23,1322524800,Stealth,stealth,1323216000,https://www.igdb.com/themes/stealth,6c0cc20c-b7fc-0007-8ae0-4cc91301e9f1
8,27,1322524800,Comedy,comedy,1323216000,https://www.igdb.com/themes/comedy,2d128835-a3c1-7979-d561-005e651ada8e
9,28,1322524800,Business,business,1323216000,https://www.igdb.com/themes/business,b088a483-b1e9-685b-dbfb-4e37b58ef69d


In [196]:
themes.columns

Index(['id', 'created_at', 'name', 'slug', 'updated_at', 'url', 'checksum'], dtype='object')

**Takeaways**

### Import manually created data reference tables from the API documentation

Had to create tables in a spreadsheet for each reference data table. 

- sheet name is the corresponding reference table
- copied values from the api documentation website to a spreadsheet

In [260]:
# brings manually created workbook in notebook as list object
df_sheet_all = pd.read_excel('manual_reference_data_tables_for_IGDB_API.xlsx', sheet_name=None)
print(df_sheet_all)

{'age_ratings_enums_category':         name  value
0       ESRB    1.0
1       PEGI    2.0
2       CERO    3.0
3        USK    4.0
4       GRAC    5.0
5  CLASS_IND    6.0
6        ACB    7.0, 'age_ratings_enums_rating':                   name  value
0                Three    1.0
1                Seven    2.0
2               Twelve    3.0
3              Sixteen    4.0
4             Eighteen    5.0
5                   RP    6.0
6                   EC    7.0
7                    E    8.0
8                  E10    9.0
9                    T   10.0
10                   M   11.0
11                  AO   12.0
12              CERO_A   13.0
13              CERO_B   14.0
14              CERO_C   15.0
15              CERO_D   16.0
16              CERO_Z   17.0
17               USK_0   18.0
18               USK_6   19.0
19              USK_12   20.0
20              USK_18   21.0
21            GRAC_ALL   22.0
22         GRAC_Twelve   23.0
23        GRAC_Fifteen   24.0
24       GRAC_Eighteen   25.0


In [261]:
# shows the sheetneames in the notebook
 df_sheet_all.keys()

dict_keys(['age_ratings_enums_category', 'age_ratings_enums_rating', 'age_ratings_descriptions_enums_', 'character_enums_gender', 'character_enums_species', 'company_enums_change_date_categ', 'company_enums_start_date_catego', 'external_game_enums_category', 'external_game_enums_media', 'company_website_enums_category', 'game_enums_category', 'game_enums_status', 'game_version_feature_eums_categ', 'game_feature_value_enums_includ', 'platform_enums_category', 'platform_version_release_date_c', 'platform_version_release_date_r', 'release_date_category', 'release_date_region', 'platform_website_enums_category', 'website_enums_category'])

In [272]:
sheetnames = ['age_ratings_enums_category', 'age_ratings_enums_rating', 'age_ratings_descriptions_enums_', 'character_enums_gender', 'character_enums_species', 'company_enums_change_date_categ', 'company_enums_start_date_catego', 'external_game_enums_category', 'external_game_enums_media', 'company_website_enums_category', 'game_enums_category', 'game_enums_status', 'game_version_feature_eums_categ', 'game_feature_value_enums_includ', 'platform_enums_category', 'platform_version_release_date_c', 'platform_version_release_date_r', 'release_date_category', 'release_date_region', 'platform_website_enums_category', 'website_enums_category']

for sheet in sheetnames:
    

In [269]:
reference_tables = pd.DataFrame(df_sheet_all.keys(), columns=['sheetname'])
reference_tables

,sheetname
0,age_ratings_enums_category
1,age_ratings_enums_rating
2,age_ratings_descriptions_enums_
3,character_enums_gender
4,character_enums_species
5,company_enums_change_date_categ
6,company_enums_start_date_catego
7,external_game_enums_category
8,external_game_enums_media
9,company_website_enums_category


In [16]:
age_ratings_enums_rating = pd.read_excel('manual_reference_data_tables_for_IGDB_API.xlsx', sheet_name='age_ratings_enums_rating')
age_ratings_enums_rating

,name,value
0,Three,1.0
1,Seven,2.0
2,Twelve,3.0
3,Sixteen,4.0
4,Eighteen,5.0
5,RP,6.0
6,EC,7.0
7,E,8.0
8,E10,9.0
9,T,10.0


## Prepare

### gamelibrary

In [17]:
game_library.drop(columns=['artworks',  'cover', 'created_at', 'external_games', 'screenshots', 'similar_games',  'tags', 'updated_at', 'url', 'videos', 'websites', 'checksum',  'alternative_names',  'keywords', 'status', 'rating_count', 'storyline', 'total_rating_count',  'hypes', 'follows', 'aggregated_rating_count'], inplace=True)
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,5.364576e+08,[5],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1.404346e+09,[9],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1.527120e+09,"[31, 32]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[1],[2],[1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1.525910e+09,[32],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[1],[3],[18],[837],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,1.422490e+09,"[15, 25]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[1, 2, 3]",[2],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,1.663805e+09,"[31, 32]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[1],[3],"[1, 23]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,1.600387e+09,"[5, 32, 33]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[1],[3],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,1.436746e+09,"[13, 15, 32]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[1],[3],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,3484.0,NaN,[119104],NaN,NaN,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# fill nulls with 0
game_library["parent_game"].fillna(0, inplace = True)
game_library["franchise"].fillna(0, inplace = True)
game_library["collection"].fillna(0, inplace = True)

In [19]:
# convert columns from float to integer
game_library['parent_game'] = game_library['parent_game'].astype(int)
game_library['franchise'] = game_library['franchise'].astype(int)
game_library['collection'] = game_library['collection'].astype(int)

In [55]:
game_library['first_release_date'] = pd.to_datetime(game_library['first_release_date'],unit='s')

In [249]:
#game_library['first_release_date'] = game_library['first_release_date'].apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [246]:
game_library[game_library['first_release_date']]

KeyError: "None of [Float64Index([         nan,  536457600.0, 1404345600.0, 1527120000.0,          nan, 1525910400.0,          nan, 1457740800.0,          nan,          nan,\n              ...\n              1618876800.0, 1416268800.0, 1512432000.0, 1543968000.0, 1582675200.0, 1473724800.0, 1459814400.0, 1550188800.0, 1393891200.0, 1384819200.0], dtype='float64', length=204500)] are in the [columns]"

In [56]:
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[5],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[9],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[31, 32]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[1],[2],[1],NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[32],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[1],[3],[18],[837],NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[15, 25]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[1, 2, 3]",[2],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,0,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[31, 32]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[1],[3],"[1, 23]",NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[5, 32, 33]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[1],[3],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,0,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[13, 15, 32]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[1],[3],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,3484,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare age ratings

In [270]:
game_library

,id,category,genres,name,parent_game,slug,summary,themes,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,aggregated_rating,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,1,"[strategy, turn-based-strategy-tbs]",Panzer Corps 2: Axis Operations - 1943,27716,panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,[22],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,0,[shooter],Dogou Souken,0,dogou-souken,An overhead-view shoot'em up game.,NaN,5.364576e+08,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,0,[puzzle],City Mysteries,0,city-mysteries,"Search buildings, streets, landmarks as you se...",NaN,1.404346e+09,[37],[138333],[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,0,"[adventure, indie]",Sword of the Black Stone,0,sword-of-the-black-stone,If your looking for action and adventure then ...,[1],1.527120e+09,[6],[188385],NaN,"[155850, 155852]",[1],[2],NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,0,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,0,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19839,3,"[shooter, role-playing-rpg, adventure]",BioShock: The Collection,0,bioshock-the-collection,BioShock: The Collection is a game bundle cons...,"[1, 17, 18, 19, 22]",1.473725e+09,"[6, 48, 49, 130]","[53184, 53185, 53202, 53203, 53204, 101896, 10...","[27442, 27443, 67844]","[185037, 185038, 185039]",[1],[1],NaN,86.974005,85.594146,NaN,NaN,1,[1686],84.214286,NaN,[7441],NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,18403,8,"[puzzle, simulator, strategy, indie]",Fantastic Contraption,168671,fantastic-contraption,Fantastic Contraption was originally a buildin...,[40],1.459814e+09,"[6, 48]","[49635, 120189, 310187]","[15194, 45964]","[148183, 148184]",[1],"[1, 7]",NaN,70.000000,68.250000,NaN,NaN,0,NaN,66.500000,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,113111,0,"[shooter, adventure]",Far Cry New Dawn,0,far-cry-new-dawn,Dive into a transformed vibrant post-apocalypt...,"[1, 38]",1.550189e+09,"[6, 48, 49, 170]","[161910, 161911, 161912, 239673]","[23926, 51062, 91551, 91702]",[74616],"[1, 2, 3]",[1],[10],66.637386,69.500511,NaN,NaN,103,NaN,72.363636,NaN,[14622],[146032],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,5595,10,"[fighting, platform, real-time-strategy-rts, s...",Awesomenauts Assemble!,1281,awesomenauts-assemble-8dcd2c21-2500-454e-950a-...,Awesomenauts Assemble! is a conversion of the ...,"[1, 18, 39]",1.393891e+09,"[48, 49]","[126067, 126068, 135865]","[15849, 32904]","[57068, 68748]","[1, 2, 3]",[4],NaN,90.000000,90.000000,NaN,NaN,820,NaN,NaN,"[90661, 90662, 90983]",[7988],NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
age_ratings

,id,category,rating,checksum,content_descriptions,synopsis
0,22474,1,8,361fd414-94d8-d6f8-6641-47abd3e8a75c,NaN,NaN
1,43430,1,8,5970434e-a37f-d43d-8975-2b23ab3370b3,NaN,NaN
2,2853,1,10,2d21809b-c282-95ba-69fb-5eda4d684b43,"[4244, 4245, 4246, 4247]",NaN
3,32550,1,9,40557417-e2a9-9a4f-5810-4159463467a1,[32972],NaN
4,65037,3,15,9c0f7e9a-8ceb-27f3-3514-cc6407a176cd,"[66283, 66284, 66285]",NaN
...,...,...,...,...,...,...
261,92085,4,19,cf63d709-d55b-fb4c-7a70-1bcdaae55d7d,NaN,NaN
262,92087,2,2,68e0ea9a-3730-364d-e2ad-d067f8cf54d7,"[96108, 96109]",NaN
263,92088,3,13,fbe60226-d930-c21a-14f7-533e01b4abf7,NaN,NaN
264,92086,1,8,11bc1676-e4fa-181d-da66-3952076e5009,[96107],NaN


In [275]:
age_ratings_enums_rating

,name,value
0,Three,1
1,Seven,2
2,Twelve,3
3,Sixteen,4
4,Eighteen,5
5,RP,6
6,EC,7
7,E,8
8,E10,9
9,T,10


In [27]:
# dataframe merging age ratings and age ratings enuns ratings. will need to replace context description with text values
age_ratings_updated = pd.merge(age_ratings, age_ratings_enums_rating, how='left', left_on = 'rating', right_on = 'value').drop(columns=[ 'category',  'checksum',  'synopsis', 'value'])
age_ratings_updated

,id,rating,content_descriptions,name
0,22474,8,NaN,E
1,43430,8,NaN,E
2,2853,10,"[4244, 4245, 4246, 4247]",T
3,32550,9,[32972],E10
4,65037,15,"[66283, 66284, 66285]",CERO_C
...,...,...,...,...
70122,93004,15,[96991],CERO_C
70123,93009,17,[96997],CERO_Z
70124,93011,13,NaN,CERO_A
70125,93010,14,"[96998, 96999, 97000]",CERO_B


In [64]:
age_rating_desc

,id,category,description,checksum
0,47,21.0,Strong Language,befd2777-5035-29b9-a592-3e0ba572b3bb
1,20,21.0,Strong Language,a0f5a6b5-3f93-00bc-f733-c5480af42132
2,50,21.0,Strong Language,21d63306-18a3-73e9-1a02-922431d31b0f
3,35,3.0,Blood,18821468-e01a-5629-67f8-36539a6cc69a
4,39,21.0,Strong Language,24807756-487a-29aa-8858-7781e3ce5e06
...,...,...,...,...
55,97124,68.0,Sexuality,19f86aca-7716-c00c-cc88-df40066dde0b
56,97122,68.0,Sexuality,19f86aca-7716-c00c-cc88-df40066dde0b
57,97123,69.0,Violence,950c5977-052d-1889-eb85-ddd363f82fa5
58,97125,69.0,Violence,950c5977-052d-1889-eb85-ddd363f82fa5


In [65]:
age_rating_desclist = age_rating_desc[['id' , 'description']].sort_values(by='id').reset_index(drop=True)
age_rating_desclist

,id,description
0,1,NaN
1,2,NaN
2,3,Strong Language
3,4,Blood
4,5,Violence
...,...,...
94055,97121,Violence
94056,97122,Sexuality
94057,97123,Violence
94058,97124,Sexuality


In [67]:
age_rating_desc_dict = age_rating_desclist.set_index('id').to_dict()['description']
age_rating_desc_dict

{1: nan,
 2: nan,
 3: 'Strong Language',
 4: 'Blood',
 5: 'Violence',
 6: 'Animated Blood',
 7: nan,
 8: 'Blood and Gore',
 9: 'Intense Violence',
 10: 'Sexual Themes',
 11: 'Strong Language',
 12: 'Use of Drugs',
 13: 'Animated Blood',
 14: nan,
 15: 'Blood and Gore',
 16: 'Intense Violence',
 17: 'Strong Language',
 18: 'Blood and Gore',
 19: 'Intense Violence',
 20: 'Strong Language',
 21: 'Blood and Gore',
 22: 'Intense Violence',
 23: 'Strong Language',
 24: 'Blood',
 25: 'Violence',
 26: 'Blood',
 27: 'Violence',
 28: 'Blood',
 29: 'Partial Nudity',
 30: 'Strong Language',
 31: 'Violence',
 35: 'Blood',
 36: 'Strong Language',
 37: 'Violence',
 38: 'Blood',
 39: 'Strong Language',
 40: 'Violence',
 41: 'Blood',
 42: 'Strong Language',
 43: 'Violence',
 44: 'Blood',
 45: 'Intense Violence',
 46: 'Sexual Content',
 47: 'Strong Language',
 48: 'Blood',
 49: 'Sexual Themes',
 50: 'Strong Language',
 51: 'Violence',
 52: 'Blood',
 53: 'Language',
 54: 'Violence',
 55: 'Blood',
 56: 'I

In [68]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [age_rating_desc_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [69]:
age_ratings_updated['content_descriptions'] = pd.DataFrame(age_ratings_updated.content_descriptions.apply(test_funct3))
age_ratings_updated

,id,rating,content_descriptions,name
0,22474,8,Not available,E
1,43430,8,Not available,E
2,2853,10,"[Blood, Drug Reference, Mild Language, Mild Vi...",T
3,32550,9,[Fantasy Violence],E10
4,65037,15,"[Violence, Crime, Languages and others]",CERO_C
...,...,...,...,...
70122,93004,15,[Sexual Content],CERO_C
70123,93009,17,[Violence],CERO_Z
70124,93011,13,Not available,CERO_A
70125,93010,14,"[Crime, Sexual Content, Violence]",CERO_B


In [78]:
age_ratings_updated[age_ratings_updated['id']== 66244 ]

,id,rating,content_descriptions,name
13214,66244,8,[Mild Fantasy Violence],E


In [79]:
age_ratings_updated[age_ratings_updated['id']== 44561 ]

,id,rating,content_descriptions,name
841,44561,1,Not available,Three


In [80]:
age_ratings_updated[age_ratings_updated['id']== 24829 ]

,id,rating,content_descriptions,name
5288,24829,8,Not available,E


In [81]:
age_ratings_updated[age_ratings_updated['id']== 34200 ]

,id,rating,content_descriptions,name
3909,34200,1,Not available,Three


In [82]:
age_ratings_updated[age_ratings_updated['id']== 56692 ]

,id,rating,content_descriptions,name
51430,56692,13,Not available,CERO_A


In [72]:
game_library.age_ratings.value_counts()

[37944]                  1
[58592]                  1
[66238]                  1
[44561, 66244]           1
[59704]                  1
                        ..
[24829, 34200, 56692]    1
[35671, 59425]           1
[90632]                  1
[85575]                  1
[7672, 42166, 67412]     1
Name: age_ratings, Length: 44287, dtype: int64

In [70]:
game_library.head(2)

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare genres

In [101]:
genres

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b
5,10,1297555200,Racing,racing,1323216000,https://www.igdb.com/genres/racing,73c2653f-54d8-c1bd-d523-dd57fb753194
6,11,1297555200,Real Time Strategy (RTS),real-time-strategy-rts,1323216000,https://www.igdb.com/genres/real-time-strategy...,aaa36cbc-2258-8653-a461-1358df8ce445
7,12,1297555200,Role-playing (RPG),role-playing-rpg,1323216000,https://www.igdb.com/genres/role-playing-rpg,42dea3b2-7fe2-e734-91cd-f80ce62a14c3
8,13,1297555200,Simulator,simulator,1323216000,https://www.igdb.com/genres/simulator,9779772a-f08f-9e8e-bd26-70c9eecc34e8
9,14,1297555200,Sport,sport,1323216000,https://www.igdb.com/genres/sport,e598c458-3e21-7a14-50a5-ea53733ab22f


In [57]:
genreslist = genres[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [200]:
genreslist.head()

,id,slug
0,2,point-and-click
1,4,fighting
2,5,shooter
3,7,music
4,8,platform


In [112]:
game_library.head(1)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
genres_dict = genreslist.set_index('id').to_dict()['slug']
genres_dict

{2: 'point-and-click',
 4: 'fighting',
 5: 'shooter',
 7: 'music',
 8: 'platform',
 9: 'puzzle',
 10: 'racing',
 11: 'real-time-strategy-rts',
 12: 'role-playing-rpg',
 13: 'simulator',
 14: 'sport',
 15: 'strategy',
 16: 'turn-based-strategy-tbs',
 24: 'tactical',
 25: 'hack-and-slash-beat-em-up',
 26: 'quiz-trivia',
 30: 'pinball',
 31: 'adventure',
 32: 'indie',
 33: 'arcade',
 34: 'visual-novel',
 35: 'card-and-board-game',
 36: 'moba'}

In [201]:
# add 0 and description to dictionary
genres_dict["0"] = "not-available"
genres_dict

{2: 'point-and-click',
 4: 'fighting',
 5: 'shooter',
 7: 'music',
 8: 'platform',
 9: 'puzzle',
 10: 'racing',
 11: 'real-time-strategy-rts',
 12: 'role-playing-rpg',
 13: 'simulator',
 14: 'sport',
 15: 'strategy',
 16: 'turn-based-strategy-tbs',
 24: 'tactical',
 25: 'hack-and-slash-beat-em-up',
 26: 'quiz-trivia',
 30: 'pinball',
 31: 'adventure',
 32: 'indie',
 33: 'arcade',
 34: 'visual-novel',
 35: 'card-and-board-game',
 36: 'moba',
 '0': 'not-available'}

In [59]:
def test_funct2(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [60]:
game_library['genres'] = pd.DataFrame(game_library.genres.apply(test_funct2))
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[1],[2],[1],NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[1],[3],[18],[837],NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[1, 2, 3]",[2],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,0,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[1],[3],"[1, 23]",NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[1],[3],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,0,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[1],[3],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,3484,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
game_library.genres.value_counts()

Not available                                                                      45591
[indie]                                                                            15157
[adventure, indie]                                                                 11240
[adventure]                                                                         9610
[simulator]                                                                         6953
                                                                                   ...  
[platform, puzzle, role-playing-rpg, turn-based-strategy-tbs, adventure, indie]        1
[role-playing-rpg, strategy, visual-novel, card-and-board-game]                        1
[point-and-click, platform, puzzle, adventure, arcade]                                 1
[shooter, role-playing-rpg, strategy, tactical, adventure]                             1
[platform, puzzle, racing, role-playing-rpg, adventure, indie]                         1
Name: genres, Length:

### prepare companies

In [232]:
companieslist = companies[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [233]:
companies_dict = companieslist.set_index('id').to_dict()['slug']

In [234]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [companies_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [ ]:
game_library['companies'] = pd.DataFrame(game_library.genres.apply(test_funct))

In [237]:
game_library.head(5)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,aggregated_rating,aggregated_rating_count,dlcs,multiplayer_modes,bundles,franchise,expansions,ports,expanded_games,remakes,remasters,standalone_expansions,forks
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[strategy, turn-based-strategy-tbs]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[shooter],Dogou Souken,NaN,NaN,"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],NaN,1604620800,https://www.igdb.com/games/dogou-souken,NaN,NaN,5e7446ff-58d9-e744-f0d3-5e10241304c1,5.364576e+08,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[puzzle],City Mysteries,NaN,"[155025, 155026, 155027]","[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],NaN,1641425677,https://www.igdb.com/games/city-mysteries,NaN,NaN,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,1.404346e+09,[37],[138333],[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[adventure, indie]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]","[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",[1],1643857546,https://www.igdb.com/games/sword-of-the-black-...,NaN,[84617],0b575316-96dc-b056-86e0-c958c1c95efb,1.527120e+09,[6],[188385],NaN,"[155850, 155852]",[1],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]","[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",NaN,1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,NaN,[192858],9585e273-2606-970f-3088-60261ea77068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[63465],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
involved_companies[involved_companies['game']== 99234]

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
322,155852,39021,1636061860,True,99234,False,False,False,1636061864,a43b0818-5e32-7bc5-9390-0bf3dbbe097c
324,155850,39021,1636061801,True,99234,False,False,False,1636061864,3d822d0f-c2de-4b52-fec2-b659b26a4153


In [239]:
companies[companies['id']== 39021]

,id,change_date_category,country,created_at,description,developed,logo,name,slug,start_date,start_date_category,updated_at,url,websites,checksum,published,change_date,changed_company_id,parent
296,39021,7,NaN,1636061132,N J FOX is an indie developer and publisher of...,NaN,NaN,N J FOX,n-j-fox,NaN,7,1636061337,https://www.igdb.com/companies/n-j-fox,NaN,91ba56c7-ee65-a0bf-174d-34ff7fd97c76,NaN,NaN,NaN,NaN


In [231]:
companies.head()

,id,change_date_category,country,created_at,description,developed,logo,name,slug,start_date,start_date_category,updated_at,url,websites,checksum,published,change_date,changed_company_id,parent
0,13710,7,752.0,1508976000,Tomolo Games develope fun and simple mobile ga...,"[74974, 74975, 74977]",1396.0,Tomolo Games,tomolo-games,1.494634e+09,0,1508976000,https://www.igdb.com/companies/tomolo-games,[1089],656407de-863f-13bc-82c0-16079a13c964,NaN,NaN,NaN,NaN
1,15094,7,233.0,1520899200,NaN,[26472],NaN,ZA/UM,za-slash-um,NaN,7,1573948800,https://www.igdb.com/companies/za-slash-um,"[1944, 3431]",d32b01ce-2272-3bf2-e809-1f16cf5c86d9,[26472],NaN,NaN,NaN
2,17841,7,246.0,1548720000,NaN,[114715],NaN,Moido Games,moido-games,NaN,7,1548720000,https://www.igdb.com/companies/moido-games,[1683],e48ef63d-fdc0-35a2-bbc6-db61277e2c25,NaN,NaN,NaN,NaN
3,16305,7,208.0,1535673600,"PortaPlay is an independent developer, residin...",[107175],1748.0,PortaPlay,portaplay,1.181952e+09,0,1536019200,https://www.igdb.com/companies/portaplay,[1323],5d28752a-62f8-f617-f0d5-6ac59c0efa6e,NaN,NaN,NaN,NaN
4,4270,7,392.0,1418428800,T's Music (ティーズ ミュージック Tiizu Myuujikku) is a g...,NaN,3304.0,T's Music,ts-music,-6.213568e+10,2,1576627200,https://www.igdb.com/companies/ts-music,[3517],238c892a-7f60-78f2-415a-7f114ccda898,NaN,NaN,NaN,NaN


### prepare collections

In [88]:
game_library.collection.value_counts()

0       171740
2986       213
3061       208
39         130
2693        98
         ...  
2785         1
5671         1
5715         1
8197         1
6542         1
Name: collection, Length: 6994, dtype: int64

In [92]:
collections[collections['id']== 2693]

,id,created_at,games,name,slug,updated_at,url,checksum
313,2693,1493798016,"[28409, 28412, 28416, 28418, 28451, 28452, 285...",ACA NEOGEO,aca-neogeo,1663671826,https://www.igdb.com/collections/aca-neogeo,a287aa1e-d85d-2c8b-a12e-9c285614faff


In [85]:
collections

,id,created_at,games,name,slug,updated_at,url,checksum
0,8202,1661995850,[189115],Grey Heritage,grey-heritage,1662330684,https://www.igdb.com/collections/grey-heritage,2e258dc4-d204-b8a8-df3a-29451234bc6b
1,7852,1651653419,[109972],Elsword,elsword,1651653420,https://www.igdb.com/collections/elsword,fe3864e8-5e0c-f27c-07cd-5ce3e4b3ffcf
2,4117,1507934083,"[11933, 18423, 39908, 61311, 61312, 61314]",Sprint,sprint,1507934083,https://www.igdb.com/collections/sprint,d0e1c01e-5c30-6ddb-2969-033fb82966f5
3,3247,1507276944,"[59261, 59263, 59264]",Hitozuma Unyu Saimin NTR Ben,hitozuma-unyu-saimin-ntr-ben,1507276944,https://www.igdb.com/collections/hitozuma-unyu...,fca09763-036f-9276-a361-2bb1b02365eb
4,5297,1516127813,"[77413, 77414, 77415, 77416, 77417]",LavaCat,lavacat,1516127813,https://www.igdb.com/collections/lavacat,34ee825d-4905-8420-5e0b-6400fe5c4826
...,...,...,...,...,...,...,...,...
318,3061,1506587780,"[67197, 67198, 68312, 68313, 68314, 68332, 683...",Arcade Archives,arcade-archives,1663676192,https://www.igdb.com/collections/arcade-archives,d9c0b1c2-cbc1-24ee-109d-8cfdea924c09
319,8281,1663322592,[207041],Hitsuji Shitsuji,hitsuji-shitsuji,1663329271,https://www.igdb.com/collections/hitsuji-shitsuji,136fdf3b-577d-bb95-963f-ac9346ba92a2
320,206,1321478702,"[843, 981, 1963, 1964, 1966, 1967, 1968, 2925,...",Rayman,rayman,1663754450,https://www.igdb.com/collections/rayman,9e989bcc-c35e-2491-03f8-a193f092ee15
321,39,1300365297,"[359, 385, 386, 387, 388, 392, 393, 395, 397, ...",Final Fantasy,final-fantasy,1663689053,https://www.igdb.com/collections/final-fantasy,ea3fd74e-18d0-dc93-c892-de2dd2934133


In [93]:
collectionslist = collections[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
collectionslist

,id,slug
0,1,bioshock
1,2,thief
2,3,fallout
3,4,dead-space
4,5,monkey-island
...,...,...
7318,8303,pocket-club
7319,8304,robbery-bob
7320,8305,pocket-size
7321,8306,superplay


In [94]:
collections_dict = collectionslist.set_index('id').to_dict()['slug']
collections_dict

{1: 'bioshock',
 2: 'thief',
 3: 'fallout',
 4: 'dead-space',
 5: 'monkey-island',
 6: 'the-elder-scrolls',
 7: 'baldur-s-gate',
 8: 'space-quest',
 9: 'deus-ex',
 10: 'dragon-age',
 11: 'dune',
 12: 'need-for-speed',
 13: 'dungeon-keeper',
 14: 'x-com',
 15: 'vampire-the-masquerade',
 17: 'devil-may-cry',
 18: 'assassin-s-creed',
 22: 'mortal-kombat',
 23: 'star-wars-knights-of-the-old-republic',
 24: 'diablo',
 25: 'mass-effect',
 26: 'left-4-dead',
 27: 'warcraft',
 28: 'star-wars-battlefront',
 29: 'star-wars-jedi-knight',
 30: 'warhammer',
 31: 'neverwinter-nights',
 32: 'mdk',
 33: 'age-of-empires',
 34: 'battlefield',
 35: 'half-life',
 36: 'crysis',
 37: 'quake',
 38: 'command-conquer',
 39: 'final-fantasy',
 40: 'swat',
 42: 'hidden-dangerous',
 43: 'might-and-magic',
 44: 'metal-gear',
 46: 'call-of-juarez',
 47: 'wolfenstein',
 48: 'total-war',
 49: 'deer-hunter',
 50: 'the-chronicles-of-riddick',
 51: 'operation-flashpoint',
 52: 'starcraft',
 53: 'simon-the-sorcerer',
 54:

In [95]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [collections_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [96]:
game_library['collection'] = pd.DataFrame(game_library.collection.apply(test_funct4))
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[1],[2],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[1],[3],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[1, 2, 3]",[2],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[1],[3],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[1],[3],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[1],[3],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,Not available,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare game modes

In [99]:
game_modes

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,Single player,single-player,1323216000,https://www.igdb.com/game_modes/single-player,a43b1688-f968-3541-0897-9735ffde1745
1,2,1298937600,Multiplayer,multiplayer,1323216000,https://www.igdb.com/game_modes/multiplayer,288b548c-11e4-d910-f037-21d4e6a33b38
2,3,1298937600,Co-operative,co-operative,1323216000,https://www.igdb.com/game_modes/co-operative,e8329d55-33e6-f7ae-ae09-2848cb7ccc90
3,4,1298937600,Split screen,split-screen,1323216000,https://www.igdb.com/game_modes/split-screen,98247b75-6f46-7c77-521f-0945c684d842
4,5,1298937600,Massively Multiplayer Online (MMO),massively-multiplayer-online-mmo,1323216000,https://www.igdb.com/game_modes/massively-mult...,307d1126-6e3b-9215-06f1-10c8ecce05b4
5,6,1588809600,Battle Royale,battle-royale,1588809600,https://www.igdb.com/game_modes/battle-royale,bfc388ac-0502-ba05-af25-3dcc94a0acf3


In [101]:
game_modeslist = game_modes[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [102]:
game_modes_dict = game_modeslist.set_index('id').to_dict()['slug']

In [103]:
def test_funct5(random_list):
    if type(random_list) == list:
        return [game_modes_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [104]:
game_library['game_modes'] = pd.DataFrame(game_library.game_modes.apply(test_funct5))

In [105]:
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[2],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[3],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[2],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[3],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[3],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[single-player],[3],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,Not available,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare player perspectives

In [108]:
player_perspectives

,id,created_at,name,slug,updated_at,url,checksum
0,1,1298937600,First person,first-person,1323216000,https://www.igdb.com/player_perspectives/first...,4e23cb22-7a70-effb-b8e1-151317c6cdbd
1,2,1298937600,Third person,third-person,1323216000,https://www.igdb.com/player_perspectives/third...,2788b856-580c-66d0-bef3-d6169034f175
2,3,1298937600,Bird view / Isometric,bird-view-slash-isometric,1582502400,https://www.igdb.com/player_perspectives/bird-...,83c59132-6edd-150f-a25f-a86ddf6a0da3
3,5,1321228800,Text,text,1323216000,https://www.igdb.com/player_perspectives/text,e2df3906-bca8-e0d1-ca3b-67e1c5cfb334
4,4,1298937600,Side view,side-view,1323216000,https://www.igdb.com/player_perspectives/side-...,2d038ac6-36c5-af5b-83fd-11a88a638631
5,7,1462233600,Virtual Reality,virtual-reality,1462233600,https://www.igdb.com/player_perspectives/virtu...,963f8a54-d40f-1c9d-3bf4-d15267043693
6,6,1413158400,Auditory,auditory,1582502400,https://www.igdb.com/player_perspectives/auditory,74528cf1-64c3-cd78-18df-fae1594ee664


In [109]:
player_perspectiveslist = player_perspectives[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [110]:
player_perspectives_dict = player_perspectiveslist.set_index('id').to_dict()['slug']

In [111]:
def test_funct6(random_list):
    if type(random_list) == list:
        return [player_perspectives_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [112]:
game_library['player_perspectives'] = pd.DataFrame(game_library.player_perspectives.apply(test_funct6))
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[52],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[37],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[6],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[82],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[6, 9, 48, 49]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[third-person],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[6, 48, 49, 130, 167, 169]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[bird-view-slash-isometric],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[6, 48, 49, 130, 167]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[bird-view-slash-isometric],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[3, 6, 14, 34, 39, 48, 130]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[single-player],[bird-view-slash-isometric],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,Not available,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare platforms

In [115]:
platforms

,id,alternative_name,category,created_at,name,platform_logo,slug,updated_at,url,versions,websites,checksum,generation,platform_family,abbreviation,summary
0,158,Commodore Dynamic Total Vision,6.0,1510012800,Commodore CDTV,292.0,commodore-cdtv,1522972800,https://www.igdb.com/platforms/commodore-cdtv,[223],[26],5d48648e-283c-e2df-df7d-6d3cbc7e0a58,NaN,NaN,NaN,NaN
1,339,Kids Computer Pico,1.0,1595808000,Sega Pico,NaN,sega-pico,1595808000,https://www.igdb.com/platforms/sega-pico,[456],NaN,6e867a5b-a071-fc7b-945b-a0622f64a4bc,4.0,3.0,NaN,NaN
2,8,PS2,1.0,1297555200,PlayStation 2,254.0,ps2,1414972800,https://www.igdb.com/platforms/ps2,"[58, 114]",NaN,be3e025c-d4ed-b795-443c-a8cb55fbfe17,6.0,1.0,PS2,NaN
3,39,NaN,4.0,1317686400,iOS,248.0,ios,1391644800,https://www.igdb.com/platforms/ios,[43],NaN,fefe4a2b-7c90-0e89-e811-902ea1cf2b58,NaN,NaN,iOS,NaN
4,94,NaN,6.0,1414195200,Commodore Plus/4,310.0,c-plus-4,1468454400,https://www.igdb.com/platforms/c-plus-4,[108],NaN,ea138789-6bbe-bcca-dff5-7e20e762f65a,NaN,NaN,C+4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,415,NaN,5.0,1660319701,Watara/QuickShot Supervision,691.0,watara-slash-quickshot-supervision,1660326748,https://www.igdb.com/platforms/watara-slash-qu...,[555],[314],4ed0c0ee-3103-4d53-c0dc-612a391de28d,4.0,NaN,NaN,NaN
193,414,LeapTV VCD,1.0,1660314942,LeapTV,690.0,leaptv,1660326997,https://www.igdb.com/platforms/leaptv,[554],[313],74a35c87-865a-52ee-29f0-70c03039ff84,8.0,NaN,NaN,NaN
194,412,Leapster Learning Game System,5.0,1660246910,Leapster,689.0,leapster,1660327194,https://www.igdb.com/platforms/leapster,[552],[311],2a4b5882-ce1b-2ba6-f4c2-0f3e55fda0b9,6.0,NaN,NaN,NaN
195,438,NaN,5.0,1663685343,Arduboy,NaN,arduboy,1663685354,https://www.igdb.com/platforms/arduboy,[588],[337],54e5a3b8-49fa-e0b3-c5e7-b7dce5ef22ba,8.0,NaN,NaN,NaN


In [116]:
platformslist = platforms[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)

In [117]:
platforms_dict = platformslist.set_index('id').to_dict()['slug']

In [118]:
def test_funct7(random_list):
    if type(random_list) == list:
        return [platforms_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [122]:
platforms_dict[92] = 'not_available'

In [123]:
platforms_dict

{3: 'linux',
 4: 'n64',
 5: 'wii',
 6: 'win',
 7: 'ps',
 8: 'ps2',
 9: 'ps3',
 11: 'xbox',
 12: 'xbox360',
 13: 'dos',
 14: 'mac',
 15: 'c64',
 16: 'amiga',
 18: 'nes',
 19: 'snes',
 20: 'nds',
 21: 'ngc',
 22: 'gbc',
 23: 'dc',
 24: 'gba',
 25: 'acpc',
 26: 'zxs',
 27: 'msx',
 29: 'smd',
 30: 'sega32',
 32: 'saturn',
 33: 'gb',
 34: 'android',
 35: 'gamegear',
 37: '3ds',
 38: 'psp',
 39: 'ios',
 41: 'wiiu',
 42: 'ngage',
 44: 'zod',
 46: 'psvita',
 47: 'vc',
 48: 'ps4--1',
 49: 'xboxone',
 50: '3do',
 51: 'fds',
 52: 'arcade',
 53: 'msx2',
 55: 'mobile',
 57: 'wonderswan',
 58: 'sfam',
 59: 'atari2600',
 60: 'atari7800',
 61: 'lynx',
 62: 'jaguar',
 63: 'atari-st',
 64: 'sms',
 65: 'atari8bit',
 66: 'atari5200',
 67: 'intellivision',
 68: 'colecovision',
 69: 'bbcmicro',
 70: 'vectrex',
 71: 'vic-20',
 72: 'ouya',
 73: 'blackberry',
 74: 'winphone',
 75: 'appleii',
 77: 'x1',
 78: 'segacd',
 79: 'neogeomvs',
 80: 'neogeoaes',
 82: 'browser',
 84: 'sg1000',
 85: 'donner30',
 86: 'turb

In [124]:
game_library['platforms'] = pd.DataFrame(game_library.platforms.apply(test_funct7))
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions
0,40104,0,1987-01-01,[shooter],Dogou Souken,[arcade],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[3ds],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[win],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[browser],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[win, ps3, ps4--1, xboxone]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[third-person],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[win, ps4--1, xboxone, switch, ps5, series-x]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[bird-view-slash-isometric],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[win, ps4--1, xboxone, switch, ps5]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[bird-view-slash-isometric],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[linux, win, mac, android, ios, ps4--1, switch]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[single-player],[bird-view-slash-isometric],"[17, 27, 28]",NaN,69.911626,74.955813,NaN,NaN,Not available,NaN,[119104],0,0,80.000000,NaN,[124836],NaN,NaN,NaN,NaN,NaN,NaN,NaN


### prepare multiplayer modes

In [133]:
multi_player_modes
multi_player_modes.rename(columns={'id': 'multi_player_mode_id'}, inplace=True)
multi_player_modes

,multi_player_mode_id,campaigncoop,dropin,game,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum,offlinecoopmax,onlinecoopmax,onlinemax
0,9953,False,False,92273,False,False,2.0,False,22.0,False,1b94d937-dc0c-c62d-10af-e58d42860bd8,NaN,NaN,NaN
1,1832,True,True,7153,False,True,0.0,False,12.0,False,2b09f64f-b8b9-6948-ad98-9ab9bede8e3b,2.0,0.0,0.0
2,7987,False,False,57887,False,True,2.0,False,49.0,False,56c9ee4c-aae3-ba3c-ead0-9e899e1c5267,0.0,0.0,0.0
3,7,False,False,46076,False,False,30.0,False,6.0,False,a41bcf9c-976d-8690-629f-4c2737f65bf1,NaN,NaN,NaN
4,10207,False,False,31256,False,False,NaN,False,82.0,False,788009fd-9e08-5826-577b-9844575ba2d9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,20775,False,False,218204,False,False,NaN,False,6.0,False,2836d768-73f2-3700-840e-a63d86599f52,NaN,NaN,NaN
245,20774,False,False,218201,False,False,NaN,False,6.0,False,dc98c1d7-c02b-67af-44ca-c59d9d1c5a98,NaN,NaN,NaN
246,20782,False,False,201498,False,False,NaN,True,6.0,False,3d75a41b-69ba-4f72-1cf0-84ffa111092a,NaN,4.0,NaN
247,20784,False,True,218300,False,False,1.0,True,6.0,False,0066f411-3aed-d2db-7a62-214939a0b2fd,1.0,4.0,8.0


In [138]:
# fill nulls with 0 or bool lean False
game_library['campaigncoop'].fillna(False, inplace = True)
game_library['dropin'].fillna(False, inplace = True)
game_library['lancoop'].fillna(False, inplace = True)
game_library['offlinecoop'].fillna(False, inplace = True)
game_library['offlinemax'].fillna(0, inplace = True)
game_library['onlinecoop'].fillna(False, inplace = True)
game_library['splitscreen'].fillna(False, inplace = True)
game_library['offlinecoopmax'].fillna(0, inplace = True)
game_library['onlinecoopmax'].fillna(0, inplace = True)
game_library['onlinemax'].fillna(0, inplace = True)

In [137]:
multi_player_modes.columns

Index(['multi_player_mode_id', 'campaigncoop', 'dropin', 'game', 'lancoop', 'offlinecoop', 'offlinemax', 'onlinecoop', 'platform', 'splitscreen', 'checksum', 'offlinecoopmax', 'onlinecoopmax', 'onlinemax'], dtype='object')

In [135]:
# dataframe merging age ratings and age ratings enuns ratings. will need to replace context description with text values
game_library = pd.merge(game_library, multi_player_modes, how='left', left_on = 'id', right_on = 'game').drop(columns=[ 'game', 'checksum', 'multi_player_mode_id', 'platform'])
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,40104,0,1987-01-01,[shooter],Dogou Souken,[arcade],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,2014-07-03,[puzzle],City Mysteries,[3ds],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[win],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,2018-05-10,[indie],sun machine,[browser],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207027,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[win, ps3, ps4--1, xboxone]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[third-person],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,NaN,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,0.0,True,False,0.0,2.0,0.0
207028,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[win, ps4--1, xboxone, switch, ps5, series-x]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[bird-view-slash-isometric],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207029,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[win, ps4--1, xboxone, switch, ps5]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[bird-view-slash-isometric],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,NaN,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,NaN,False,False,2.0,NaN,NaN
207030,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[linux, win, mac, android, ios, ps4--1, switch]","[37079, 37080, 37081, 153404, 153405, 155118, ...",holy-potatoes-a-weapon-shop,Holy Potatoes! A Weapon Shop?! is a wacky simu...,"[22632, 61132]","[62816, 96913, 96914]",[single-play

### prepare 

In [139]:
game_library.head()

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,40104,0,1987-01-01,[shooter],Dogou Souken,[arcade],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
1,85031,0,2014-07-03,[puzzle],City Mysteries,[3ds],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[win],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
4,139538,0,2018-05-10,[indie],sun machine,[browser],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0


In [142]:
franchises

,id,created_at,games,name,slug,updated_at,url,checksum
0,545,1428781176,"[6805, 9626, 10091, 11754, 11755, 11767, 11768...",VOCALOID,vocaloid,1428781176,https://www.igdb.com/franchises/vocaloid,d7a62e0a-1662-f2e4-1dde-1fc10c829e39
1,2574,1617402728,[145390],NextRev,nextrev,1617402728,https://www.igdb.com/franchises/nextrev,ac73431c-640d-8b37-c757-09c2135e1167
2,2798,1623025704,[151890],Tuned Turtle,tuned-turtle,1623025704,https://www.igdb.com/franchises/tuned-turtle,3220ee67-be4b-ce96-dd2e-241cfa976809
3,783,1458155907,[18385],Austin Powers,austin-powers,1458155907,https://www.igdb.com/franchises/austin-powers,c2e2e449-0b19-7bc3-9ed2-ceba6a5844aa
4,730,1446474129,NaN,Kane & Lynch,kane-and-lynch,1446474129,https://www.igdb.com/franchises/kane-and-lynch,81914463-391d-da66-9f96-3cd8fb085c2c
...,...,...,...,...,...,...,...,...
200,4006,1662655852703,NaN,Joey and Penguin,joey-and-penguin,1662655852,https://www.igdb.com/franchises/joey-and-penguin,cc39cbfe-697b-6209-3008-113ac97833be
201,51,1372173591416,"[138, 189, 190, 390, 422, 542, 2050, 2157, 217...",LEGO,lego,1663230468,https://www.igdb.com/franchises/lego,73dbe2df-dc2e-2be9-d6cd-144dff577638
202,4032,1663324223545,NaN,LEX Studios,lex-studios,1663324223,https://www.igdb.com/franchises/lex-studios,7bac82ac-f258-f502-cc8f-1e5fd77a7079
203,4043,1663422306558,NaN,Enmity,enmity,1663422306,https://www.igdb.com/franchises/enmity,73900f48-0c47-db35-0508-002b46a13a32


In [143]:
franchiseslist = franchises[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
franchises_dict = franchiseslist.set_index('id').to_dict()['slug']

def test_funct8(random_list):
    if type(random_list) == list:
        return [franchises_dict[i] for i in random_list]
    else:  
        return 'Not available'

game_library['franchises'] = pd.DataFrame(game_library.franchises.apply(test_funct8))
game_library

,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,40104,0,1987-01-01,[shooter],Dogou Souken,[arcade],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
1,85031,0,2014-07-03,[puzzle],City Mysteries,[3ds],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[win],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
4,139538,0,2018-05-10,[indie],sun machine,[browser],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207027,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[win, ps3, ps4--1, xboxone]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[third-person],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,Not available,NaN,0,0,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,0.0,True,False,0.0,2.0,0.0
207028,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[win, ps4--1, xboxone, switch, ps5, series-x]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[bird-view-slash-isometric],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
207029,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[win, ps4--1, xboxone, switch, ps5]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[bird-view-slash-isometric],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,Not available,NaN,0,0,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,0.0,False,False,2.0,0.0,0.0
207030,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A Weapon Shop?!,"[linux, win, mac, android, ios, ps4--1, switch]","[37079, 37080, 37081, 153404, 

In [144]:
game_library.franchises.value_counts()

Not available                    189970
[star-wars]                         227
[mario]                             224
[sonic-the-hedgehog]                211
[final-fantasy]                     210
                                  ...  
[swat-kats]                           1
[brodrene-mortensens-jul]             1
[black-otaku]                         1
[nova]                                1
[arcade-archives, metal-slug]         1
Name: franchises, Length: 3281, dtype: int64

In [146]:
game_library['franchise'] = pd.DataFrame(game_library.franchise.apply(test_funct8))
game_library


,id,category,first_release_date,genres,name,platforms,release_dates,slug,summary,age_ratings,involved_companies,game_modes,player_perspectives,themes,game_engines,rating,total_rating,version_parent,version_title,collection,franchises,bundles,parent_game,franchise,aggregated_rating,multiplayer_modes,dlcs,ports,expansions,expanded_games,remakes,remasters,forks,standalone_expansions,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,40104,0,1987-01-01,[shooter],Dogou Souken,[arcade],[91007],dogou-souken,An overhead-view shoot'em up game.,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
1,85031,0,2014-07-03,[puzzle],City Mysteries,[3ds],[138333],city-mysteries,"Search buildings, streets, landmarks as you se...",[37944],"[130531, 130532]",Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
2,99234,0,2018-05-24,"[adventure, indie]",Sword of the Black Stone,[win],[188385],sword-of-the-black-stone,If your looking for action and adventure then ...,NaN,"[155850, 155852]",[single-player],[third-person],[1],NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
3,148930,0,NaT,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,Not available,NaN,bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,NaN,NaN,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
4,139538,0,2018-05-10,[indie],sun machine,[browser],[213594],sun-machine,"a story of dark days, chronic fatigue, optimis...",NaN,[117090],[single-player],[bird-view-slash-isometric],[18],[837],NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207027,19927,0,2015-01-29,"[strategy, hack-and-slash-beat-em-up]",Bladestorm: Nightmare,"[win, ps3, ps4--1, xboxone]","[101466, 101467, 101468, 101469, 101470, 10147...",bladestorm-nightmare,BLADESTORM: Nightmare is the newest evolution ...,"[13884, 32430, 68614]","[52681, 52682]","[single-player, multiplayer, co-operative]",[third-person],"[1, 17, 22, 39]",NaN,70.000000,61.142857,NaN,NaN,Not available,Not available,NaN,0,Not available,52.285714,[6919],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,0.0,True,False,0.0,2.0,0.0
207028,134823,0,2022-09-22,"[adventure, indie]",Serial Cleaners,"[win, ps4--1, xboxone, switch, ps5, series-x]","[364770, 364771, 364772, 364773, 364774, 36555...",serial-cleaners,Serial Cleaners is a single-player stealth act...,"[89069, 90047, 91617]",[146381],[single-player],[bird-view-slash-isometric],"[1, 23]",NaN,NaN,NaN,NaN,NaN,Not available,Not available,NaN,0,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,False,0.0,0.0,0.0
207029,146102,0,2020-09-18,"[shooter, indie, arcade]",Crisis Wing,"[win, ps4--1, xboxone, switch, ps5]","[240114, 309844, 312693, 315199, 327744, 338036]",crisis-wing,Crisis Wing is a mid-90’s style arcade shoot’e...,"[45884, 52500]",[125668],[single-player],[bird-view-slash-isometric],"[1, 18]",NaN,NaN,70.000000,NaN,NaN,Not available,Not available,NaN,0,Not available,70.000000,[14192],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,0.0,False,False,2.0,0.0,0.0
207030,12880,0,2015-07-13,"[simulator, strategy, indie]",Holy Potatoes! A

In [147]:
game_library.franchise.value_counts()

Not available    207032
Name: franchise, dtype: int64

reverify franchise value puling in fresh raw dataframe